# Postes km

In [199]:
import pandas as pd

postes_viejo = pd.read_excel("postes_km_anterior.xlsx")

nuevos_nombres = {
    "ruta": "Ruta",
    "km": "KM",
    "longitude": "Lat",
    "latitude": "Long"
}
postes_viejo.rename(columns=nuevos_nombres, inplace=True)

postes_viejo = postes_viejo[["Ruta","KM","Lat","Long"]]
postes_viejo

,Ruta,KM,Lat,Long
0,7,345,-54.585138,-32.575664
1,7,349,-54.570987,-32.542146
2,7,355,-54.525446,-32.513338
3,7,365,-54.441941,-32.468569
4,7,370,-54.406333,-32.435834
...,...,...,...,...
6103,200,48,-55.738036,-34.759697
6104,200,47,-55.747790,-34.763694
6105,200,50,-55.718451,-34.752856
6106,200,51,-55.707844,-34.750046


In [200]:
postes_relevados = pd.read_excel("radares_geolocalizacion.xlsx", sheet_name="Postes")
postes_relevados = postes_relevados[["Ruta","KM","Lat","Long"]]
postes_relevados

,Ruta,KM,Lat,Long
0,IB,21,-34.831003,-56.010360
1,IB,22,-34.826627,-56.000459
2,IB,24,-34.818999,-55.982176
3,IB,28,-34.798131,-55.946357
4,IB,32,-34.787967,-55.906642
...,...,...,...,...
217,1,4,-34.872734,-56.237084
218,1,3,-34.870019,-56.227237
219,1,2,-34.871604,-56.217586
220,1,100,-34.371451,-57.043962


In [201]:
postes_combinados = pd.concat([postes_viejo, postes_relevados], ignore_index=True)

postes_combinados['Ruta'] = postes_combinados['Ruta'].replace(200, 'IB')
postes_combinados_sin_duplicados = postes_combinados.drop_duplicates(subset=["Ruta", "KM"], keep="first")


postes_combinados_sin_duplicados

,Ruta,KM,Lat,Long
0,7,345,-54.585138,-32.575664
1,7,349,-54.570987,-32.542146
2,7,355,-54.525446,-32.513338
3,7,365,-54.441941,-32.468569
4,7,370,-54.406333,-32.435834
...,...,...,...,...
6319,1,52,-34.641230,-56.627981
6322,1,122,-34.334930,-57.270923
6324,1,2,-34.871735,-56.217147
6325,1,4,-34.872734,-56.237084


In [202]:
import pandas as pd

# Definir los rangos válidos para Uruguay
LAT_MIN, LAT_MAX = -35.5, -30.0
LONG_MIN, LONG_MAX = -58.5, -53.0

def corregir_coordenadas(row):
    lat, long = row['Lat'], row['Long']
    
    # Verificar si el punto está dentro del rango válido
    if LAT_MIN <= lat <= LAT_MAX and LONG_MIN <= long <= LONG_MAX:
        return lat, long  # Coordenadas correctas, no hacer nada
    
    # Rotar las coordenadas si están fuera del rango
    if LAT_MIN <= long <= LAT_MAX and LONG_MIN <= lat <= LONG_MAX:
        return long, lat  # Intercambiar latitud y longitud
    
    # Dejar como NaN si sigue fuera del rango después de rotar
    return pd.NA, pd.NA

# Aplicar la corrección al DataFrame
postes_combinados_sin_duplicados[['Lat', 'Long']] = postes_combinados_sin_duplicados.apply(
    lambda row: pd.Series(corregir_coordenadas(row)), axis=1
)

# Eliminar filas con coordenadas que siguen siendo inválidas (opcional)
postes_combinados_sin_duplicados = postes_combinados_sin_duplicados.dropna(subset=['Lat', 'Long'])

postes_combinados_sin_duplicados


C:\Users\fpino\AppData\Local\Temp\ipykernel_2264\185358471.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postes_combinados_sin_duplicados[['Lat', 'Long']] = postes_combinados_sin_duplicados.apply(


,Ruta,KM,Lat,Long
0,7,345,-32.575664,-54.585138
1,7,349,-32.542146,-54.570987
2,7,355,-32.513338,-54.525446
3,7,365,-32.468569,-54.441941
4,7,370,-32.435834,-54.406333
...,...,...,...,...
6319,1,52,-34.641230,-56.627981
6322,1,122,-34.334930,-57.270923
6324,1,2,-34.871735,-56.217147
6325,1,4,-34.872734,-56.237084


In [203]:
postes_combinados_sin_duplicados.to_excel("Postes_KM_2025.xlsx", index=False)

# Linea Base Carteles

In [104]:
carteles = pd.read_excel("radares_geolocalizacion.xlsx", sheet_name="Linea Base")
carteles

,Nombre de video,Lat,Long,Código Radar
0,GX010067,-34.709598,-56.207375,M108
1,GX010067,-34.704995,-56.202791,M108
2,GX010068,-34.601586,-56.261079,M023 A
3,GX010068,-34.593040,-56.262199,M023 B
4,GX010068,-34.591447,-56.262751,M023 B
...,...,...,...,...
244,GX010127,-33.512603,-56.913041,M056
245,GX010128,-33.515000,-56.883330,M058
246,GX010128,-33.515126,-56.884597,M058
247,GX010129,-33.536184,-56.888858,M057


In [105]:
df_agrupado = carteles.sort_values(by="Código Radar")

df_agrupado

,Nombre de video,Lat,Long,Código Radar
92,GX010024,-34.828071,-56.003778,M001 - A
91,GX010024,-34.829707,-56.007576,M001 - A
134,GX020025,-34.824717,-55.996601,M001 - B
93,GX010024,-34.804940,-55.956526,M002 - A
133,GX020025,-34.799423,-55.948591,M002 - B
...,...,...,...,...
127,GX020024,-34.766409,-55.761199,No corresponde
104,GX010025,-34.765050,-55.752932,No corresponde
95,GX010024,-34.785897,-55.889999,No corresponde
94,GX010024,-34.787301,-55.892055,No corresponde


In [106]:
df_agrupado["Código Radar"] = df_agrupado["Código Radar"].str.replace(" ", "", regex=False)
df_agrupado["Código Radar"] = df_agrupado["Código Radar"].str.upper()  

In [107]:
df_agrupado["Código Radar"].value_counts()

Código Radar
M010             12
NOCORRESPONDE     5
M018              5
M008              4
M058              4
                 ..
M064              1
M045              1
M046              1
M091              1
M092              1
Name: count, Length: 119, dtype: int64

In [108]:
df_agrupado[df_agrupado["Código Radar"]=="M010"]


,Nombre de video,Lat,Long,Código Radar
40,GX010089,-33.874620,-57.367800,M010
51,GX010091,-33.874775,-57.372222,M010
50,GX010091,-33.882646,-57.372031,M010
49,GX010091,-33.890082,-57.372166,M010
48,GX010091,-33.876946,-57.372316,M010
47,GX010091,-33.874927,-57.374667,M010
52,GX010091,-33.874497,-57.365243,M010
46,GX010091,-33.874970,-57.375892,M010
42,GX010089,-33.877623,-57.357376,M010
41,GX010089,-33.874505,-57.365159,M010


In [109]:
df_agrupado.to_excel("Carteles_ordenado.xlsx", index=False)

PermissionError: [Errno 13] Permission denied: 'Carteles_ordenado.xlsx'

comentario: la idea es usar una logica para quedarme con un sub data frame donde tengamos solo un sentido en algunos casos y doble snetido en el otro, por lo tanto deberiamos hacer este siguiente codigo y buscar los nombres qu nos gustaria o bien implementar logica donde se logre identigficar que tenemos a + y - buscando un prefijo comun y solo la variante de carcater finañl 

In [125]:
radares = pd.read_excel("radares_geolocalizacion.xlsx", sheet_name="Radares")
radares["Código Radar"] = radares["Código Radar"].str.replace(" ", "", regex=False)
radares["Código Radar"] = radares["Código Radar"].str.upper()  
radares.head(10)

,Nombre Video,Lat,Long,Código Radar
0,GX020024,-34.827427,-56.002304,M001-A
1,GX020025,-34.825792,-55.998950,M001-B
2,GX010024,-34.803354,-55.954161,M002-A
3,GX010024,-34.780609,-55.873260,M003-A
4,GX020025,-34.780124,-55.871923,M003-B
5,GX010024,-34.777975,-55.857361,M004-A
6,GX010025,-34.771885,-55.823007,M005-A
7,GX010024,-34.771719,-55.821438,M005-B
8,GX010079,-34.591356,-56.703672,M007
9,GX010082,-34.336311,-57.259966,M008


In [154]:
valores_a_filtrar = [
    "M101", "M102", "M103", "M104", "M105",
    "M107", "M108", "M109", "M110","M111",
    "M112", "M112", "M113", "M114","M115"
]


subcadenas = []
for index, row in df_agrupado.iterrows():
    subcadena_encontrada = None
    for valor in valores_a_filtrar:
        if valor in row["Código Radar"]:
            subcadena_encontrada = valor
            break
    subcadenas.append(subcadena_encontrada)

df_agrupado["Subcadena"] = subcadenas


carteles_filtrado = df_agrupado[df_agrupado["Subcadena"].notna()]

carteles_filtrado

,Nombre de video,Lat,Long,Código Radar,Subcadena
223,GX010118,-31.417612,-57.939999,M101,M101
222,GX010118,-31.417648,-57.939970,M101,M101
221,GX010118,-31.414117,-57.942704,M101,M101
30,GX010084,-34.444148,-57.837359,M102,M102
31,GX010084,-34.440489,-57.838153,M102,M102
32,GX010086,-34.034845,-57.901721,M103,M103
33,GX010086,-34.029713,-57.897523,M103,M103
34,GX010087,-33.979385,-58.292338,M104,M104
35,GX010087,-33.980959,-58.289127,M104,M104
10,GX010072,-34.424652,-56.279554,M105,M105


In [155]:
radares_filtrado = radares[radares["Código Radar"].isin(valores_a_filtrar)]

radares_filtrado

,Nombre Video,Lat,Long,Código Radar
81,GX010118,-31.415892,-57.941332,M101
82,GX010084,-34.445776,-57.837290,M102
83,GX010086,-34.032025,-57.899700,M103
84,GX010087,-33.981080,-58.288348,M104
85,GX010072,-34.425279,-56.282218,M105
86,GX010073,-34.644944,-56.068053,M107
87,GX010067,-34.704276,-56.201969,M108
88,GX010067,-34.707162,-56.204991,M108
89,GX010074,-34.643536,-56.070499,M109
90,GX010022,-34.754409,-56.019251,M110


In [149]:
radares[radares["Código Radar"].isin(["M004"])]

,Nombre Video,Lat,Long,Código Radar


In [156]:
import pandas as pd
import folium
import random
from geopy.distance import geodesic

# Crear el mapa centrado en las coordenadas promedio de `carteles_filtrado`
mapa = folium.Map(location=[carteles_filtrado["Lat"].mean(), carteles_filtrado["Long"].mean()], zoom_start=13)

# Generar colores más oscuros únicos para cada valor de "Código Radar"
valores_unicos = pd.concat([carteles_filtrado["Código Radar"], radares_filtrado["Código Radar"]]).unique()
colores = {valor: f"#{random.randint(0, 0x7F7F7F):06x}" for valor in valores_unicos}  # Colores más oscuros

# Función para encontrar el punto más cercano
def encontrar_mas_cercano(origen, puntos_disponibles):
    return min(
        puntos_disponibles,
        key=lambda destino: geodesic((origen["Lat"], origen["Long"]), (destino["Lat"], destino["Long"])).meters
    )

# Procesar cada grupo de "Código Radar"
for codigo_radar in valores_unicos:
    puntos_grupo = carteles_filtrado[carteles_filtrado["Subcadena"] == codigo_radar].copy()
    marcador_estandar = radares_filtrado[radares_filtrado["Código Radar"] == codigo_radar]
    
    if marcador_estandar.empty:
        continue

    marcador_final = {
        "Lat": marcador_estandar.iloc[0]["Lat"],
        "Long": marcador_estandar.iloc[0]["Long"]
    }
    
    # Generar las líneas conectando cada punto al más cercano
    puntos_disponibles = puntos_grupo.to_dict("records")
    if puntos_disponibles:
        punto_actual = puntos_disponibles.pop(0)  # Tomar el primer punto

        while puntos_disponibles:
            siguiente_punto = encontrar_mas_cercano(punto_actual, puntos_disponibles)
            folium.PolyLine(
                locations=[(punto_actual["Lat"], punto_actual["Long"]), (siguiente_punto["Lat"], siguiente_punto["Long"])],
                color=colores[codigo_radar],
                weight=3  # Grosor de la línea
            ).add_to(mapa)
            puntos_disponibles.remove(siguiente_punto)
            punto_actual = siguiente_punto
        
        # Conectar el marcador estándar al punto más cercano
        punto_cercano = encontrar_mas_cercano(marcador_final, puntos_grupo.to_dict("records"))
        folium.PolyLine(
            locations=[(punto_cercano["Lat"], punto_cercano["Long"]), (marcador_final["Lat"], marcador_final["Long"])],
            color=colores[codigo_radar],
            weight=3  # Grosor de la línea
        ).add_to(mapa)

    # Añadir CircleMarkers para el grupo
    for _, row in carteles_filtrado[carteles_filtrado["Subcadena"] == codigo_radar].iterrows():
        folium.CircleMarker(
            location=[row["Lat"], row["Long"]],
            radius=7,
            color=colores[codigo_radar],
            fill=True,
            fill_color=colores[codigo_radar],
            fill_opacity=0.99,
            popup=f"Código Radar: {row['Código Radar']}"
        ).add_to(mapa)

    # Añadir marcador estándar para el grupo
    folium.Marker(
        location=[marcador_final["Lat"], marcador_final["Long"]],
        popup=f"Radar estándar: {codigo_radar}",
        icon=folium.Icon(color="white", icon_color=colores[codigo_radar])
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save("mapa_geografico_9.html")
print("Mapa interactivo guardado como 'mapa_geografico.html'")


Mapa interactivo guardado como 'mapa_geografico.html'


# Muesta para desarrollo 

### MAPA CON RADARES SELECCIONADOS 

In [162]:
carteles = pd.read_excel("Radares_prueba.xlsx", sheet_name="Linea Base")
radares = pd.read_excel("Radares_prueba.xlsx", sheet_name="Radares")


valores_a_filtrar = [
    "M001-A", "M001-B", "M084-E", "M084-S", "M102", "M104", "M108"
]

subcadenas = []
for index, row in carteles.iterrows():
    subcadena_encontrada = None
    for valor in valores_a_filtrar:
        if valor in row["Código Radar"]:
            subcadena_encontrada = valor
            break
    subcadenas.append(subcadena_encontrada)

carteles["Subcadena"] = subcadenas
carteles_filtrado = carteles[carteles["Subcadena"].notna()]


radares_filtrado = radares[radares["Código Radar"].isin(valores_a_filtrar)]



# Crear el mapa centrado en las coordenadas promedio de `carteles_filtrado`
mapa = folium.Map(location=[carteles_filtrado["Lat"].mean(), carteles_filtrado["Long"].mean()], zoom_start=13)

# Generar colores más oscuros únicos para cada valor de "Código Radar"
valores_unicos = pd.concat([carteles_filtrado["Código Radar"], radares_filtrado["Código Radar"]]).unique()
colores = {valor: f"#{random.randint(0, 0x7F7F7F):06x}" for valor in valores_unicos}  # Colores más oscuros

# Función para encontrar el punto más cercano
def encontrar_mas_cercano(origen, puntos_disponibles):
    return min(
        puntos_disponibles,
        key=lambda destino: geodesic((origen["Lat"], origen["Long"]), (destino["Lat"], destino["Long"])).meters
    )

# Procesar cada grupo de "Código Radar"
for codigo_radar in valores_unicos:
    puntos_grupo = carteles_filtrado[carteles_filtrado["Subcadena"] == codigo_radar].copy()
    marcador_estandar = radares_filtrado[radares_filtrado["Código Radar"] == codigo_radar]
    
    if marcador_estandar.empty:
        continue

    marcador_final = {
        "Lat": marcador_estandar.iloc[0]["Lat"],
        "Long": marcador_estandar.iloc[0]["Long"]
    }
    
    # Generar las líneas conectando cada punto al más cercano
    puntos_disponibles = puntos_grupo.to_dict("records")
    if puntos_disponibles:
        punto_actual = puntos_disponibles.pop(0)  # Tomar el primer punto

        while puntos_disponibles:
            siguiente_punto = encontrar_mas_cercano(punto_actual, puntos_disponibles)
            folium.PolyLine(
                locations=[(punto_actual["Lat"], punto_actual["Long"]), (siguiente_punto["Lat"], siguiente_punto["Long"])],
                color=colores[codigo_radar],
                weight=3  # Grosor de la línea
            ).add_to(mapa)
            puntos_disponibles.remove(siguiente_punto)
            punto_actual = siguiente_punto
        
        # Conectar el marcador estándar al punto más cercano
        punto_cercano = encontrar_mas_cercano(marcador_final, puntos_grupo.to_dict("records"))
        folium.PolyLine(
            locations=[(punto_cercano["Lat"], punto_cercano["Long"]), (marcador_final["Lat"], marcador_final["Long"])],
            color=colores[codigo_radar],
            weight=3  # Grosor de la línea
        ).add_to(mapa)

    # Añadir CircleMarkers para el grupo
    for _, row in carteles_filtrado[carteles_filtrado["Subcadena"] == codigo_radar].iterrows():
        folium.CircleMarker(
            location=[row["Lat"], row["Long"]],
            radius=7,
            color=colores[codigo_radar],
            fill=True,
            fill_color=colores[codigo_radar],
            fill_opacity=0.99,
            popup=f"Código Radar: {row['Código Radar']}"
        ).add_to(mapa)

    # Añadir marcador estándar para el grupo
    folium.Marker(
        location=[marcador_final["Lat"], marcador_final["Long"]],
        popup=f"Radar estándar: {codigo_radar}",
        icon=folium.Icon(color="white", icon_color=colores[codigo_radar])
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save("mapa_geografico_sub-muestra.html")
print("Mapa interactivo guardado como 'mapa_geografico.html'")

Mapa interactivo guardado como 'mapa_geografico.html'


### Generación de carpeta con clips y frame 
 

In [23]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog, font
import os
from geopy.distance import geodesic
import cv2
import time
from math import radians, sin, cos, sqrt, atan2

# Función de haversine vectorizada
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radio de la Tierra en metros
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2.0) ** 2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

carteles = pd.read_excel("radares_geolocalizacion.xlsx", sheet_name="Linea Base")

radares = pd.read_excel("radares_geolocalizacion.xlsx", sheet_name="Radares")

postes = pd.read_excel("Postes_KM_2025.xlsx")
filtros = radares['Ruta'].tolist()
postes = postes[postes['Ruta'].isin(filtros)]


df = pd.read_csv('./dev/GX010022_HERO11 Black-GPS9.csv')

df['Distancia'] = df.apply(
        lambda row: haversine(
            row['GPS (Lat.) [deg]'], row['GPS (Long.) [deg]'], 
            df.iloc[row.name - 1]['GPS (Lat.) [deg]'], df.iloc[row.name - 1]['GPS (Long.) [deg]']
        ) if row.name > 0 else 0,  # La primera fila no tiene punto anterior
        axis=1
    )

df

,cts,date,GPS (Lat.) [deg],GPS (Long.) [deg],GPS (Alt.) [m],GPS (2D) [m/s],GPS (3D) [m/s],GPS (days) [deg],GPS (secs) [s],GPS (DOP) [deg],GPS (fix) [deg],altitude system,Distancia
0,1.307800e+02,2024-08-08T17:16:03.199Z,-34.726454,-55.963808,28.717,2.914,2.92,8986,62163.199,1.22,3,MSLV,0.000000
1,2.309829e+02,2024-08-08T17:16:03.298Z,-34.726455,-55.963810,28.719,2.705,2.92,8986,62163.299,1.22,3,MSLV,0.270362
2,3.311858e+02,2024-08-08T17:16:03.398Z,-34.726456,-55.963813,28.675,2.851,2.71,8986,62163.399,1.22,3,MSLV,0.291753
3,4.313887e+02,2024-08-08T17:16:03.499Z,-34.726458,-55.963816,28.649,2.917,2.85,8986,62163.499,1.22,3,MSLV,0.299522
4,5.315916e+02,2024-08-08T17:16:03.599Z,-34.726459,-55.963818,28.647,2.788,2.92,8986,62163.599,1.22,3,MSLV,0.284066
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21136,2.113656e+06,2024-08-08T17:51:16.800Z,-34.793172,-56.116254,42.401,13.822,13.86,8986,64276.800,1.07,3,MSLV,1.376243
21137,2.113747e+06,2024-08-08T17:51:16.900Z,-34.793179,-56.116267,42.417,13.899,13.82,8986,64276.900,1.07,3,MSLV,1.395632
21138,2.113838e+06,2024-08-08T17:51:17.000Z,-34.793185,-56.116280,42.420,13.827,13.90,8986,64277.000,1.07,3,MSLV,1.382026
21139,2.113929e+06,2024-08-08T17:51:17.099Z,-34.793192,-56.116293,42.429,13.825,13.83,8986,64277.100,1.07,3,MSLV,1.382026


### **Motivación del Código**

El objetivo de este bloque de código es optimizar la identificación de postes relacionados con una trayectoria geográfica específica. Esto es esencial para reducir el tamaño de datos a procesar y enfocarse únicamente en los elementos relevantes al análisis. Primero, se filtran los postes que están dentro de un radio de 1.1 km desde cuatro puntos equidistantes en la trayectoria. Luego, se identifican los postes más cercanos a menos de 20 metros para asociarlos con puntos específicos de la trayectoria. Este enfoque utiliza cálculos vectorizados con Numpy para optimizar el proceso de comparación, dado que las operaciones sobre cada par de coordenadas pueden ser intensivas en tiempo cuando se manejan grandes volúmenes de datos.

Además, no se procesa cada punto del DataFrame de la trayectoria. En lugar de eso, se avanza dinámicamente por el DataFrame, acumulando al menos 20 metros entre puntos antes de realizar cálculos adicionales. Este enfoque reduce significativamente la cantidad de información procesada y, por ende, el tiempo de ejecución. Para facilitar este avance, se utiliza una columna precomputada llamada `Distancia`, que contiene la distancia en metros entre cada punto y el anterior. Esta columna permite calcular rápidamente la distancia acumulada y controlar de manera precisa los saltos en la trayectoria.

---

### **Explicación del Código**

#### **Filtro del DataFrame `postes`**

1. **Selección de puntos equidistantes**:
   - Se eligen cuatro puntos de la trayectoria (inicio, fin y dos puntos intermedios equidistantes).
   - Esto reduce significativamente el número de cálculos, ya que en lugar de evaluar cada punto de la trayectoria, se evalúan solo estos puntos representativos.

2. **Radio de 1.1 km**:
   - Usando la fórmula de geodesic, se calcula la distancia desde cada punto equidistante a todos los postes en el DataFrame.
   - Los postes dentro de este radio se almacenan y se eliminan duplicados posteriormente.

3. **Filtrado por rutas**:
   - Se identifican las rutas de los postes encontrados y se usa este conjunto para filtrar el DataFrame original. Esto asegura que los postes que no están cerca de la trayectoria sean descartados, mejorando la eficiencia del siguiente paso.

#### **Búsqueda de Postes Más Cercanos**

1. **Uso de cálculos vectorizados**:
   - La función `calculate_distances_vectorized` utiliza Numpy para calcular las distancias entre un punto y todos los postes de manera eficiente.
   - Este enfoque evita bucles explícitos, lo que resulta en un código más rápido y escalable.

2. **Avance basado en distancias acumuladas**:
   - Se avanza dinámicamente en la trayectoria acumulando al menos 20 metros en la columna `Distancia` antes de realizar nuevos cálculos.
   - Este método permite evitar consultas innecesarias y reduce la cantidad de datos procesados, manteniendo un análisis eficiente.

3. **Identificación del poste más cercano**:
   - Para cada punto de la trayectoria, se identifica el poste más cercano dentro de un rango de 20 metros usando `np.argmin`, que encuentra el índice del valor mínimo en un array.

4. **Registro de datos relevantes**:
   - Si un poste se encuentra dentro del rango, se almacenan sus datos junto con la posición correspondiente de la trayectoria.

5. **Condiciones de finalización**:
   - Si el final del DataFrame se alcanza sin cumplir los criterios de avance, el bucle termina, asegurando que no haya ciclos infinitos.

---

### **Razón para los Cálculos Vectorizados y la Columna `Distancia`**

Los cálculos vectorizados son esenciales para evitar cuellos de botella en el procesamiento. Al usar Numpy, las operaciones matemáticas se aplican a arrays completos en lugar de realizar iteraciones sobre cada par de puntos. Esto resulta en una implementación más eficiente en términos de tiempo, especialmente cuando se manejan miles de puntos en trayectorias largas y con muchos postes.

La columna `Distancia`, que contiene las distancias en metros entre puntos consecutivos, es clave para controlar el avance en la trayectoria. Permite acumular fácilmente distancias hasta alcanzar al menos 20 metros, eliminando la necesidad de procesar cada punto del DataFrame. Esto no solo mejora el rendimiento, sino que también asegura que los cálculos se realicen de manera uniforme en términos de distancia recorrida.

---

### **Conclusión**

Este código es un ejemplo de cómo filtrar datos geográficos relevantes de manera eficiente, aprovechando tanto operaciones matemáticas optimizadas como un enfoque lógico para limitar los datos procesados. La combinación de cálculos vectorizados, el uso de la columna `Distancia` y el avance dinámico basado en metros recorridos garantiza un balance entre precisión y rendimiento.


In [24]:
#######################################################################################################################################################
############################ Filtro df postes

# Número de puntos a seleccionar (incluyendo el primero y el último)
num_puntos_interiores = 2
total_puntos = num_puntos_interiores + 2  # Incluye el primero y el último

# Calcular los índices equiespaciados
indices = []
indices.append(0)  # Primer punto

# Calcular los índices interiores equiespaciados
for i in range(1, total_puntos - 1):
    indice = round(len(df) * i / (total_puntos - 1))
    indices.append(indice)

indices.append(len(df) - 1)  # Último punto

# Asegurar que no haya índices duplicados (en caso de DataFrame pequeño)
indices = sorted(set(indices))

# Crear una lista para almacenar los postes dentro del radio de 1100 metros
postes_cercanos = []

# Iterar sobre los índices seleccionados en `df`
for idx in indices:
    punto = df.iloc[idx]
    punto_coords = (punto['GPS (Lat.) [deg]'], punto['GPS (Long.) [deg]'])

    # Calcular la distancia entre el punto actual y cada poste
    for _, poste in postes.iterrows():
        poste_coords = (poste['Lat'], poste['Long'])
        distancia = geodesic(punto_coords, poste_coords).meters

        # Si el poste está dentro del radio, añadirlo a la lista
        if distancia <= 1100:
            postes_cercanos.append(poste)

# Crear un nuevo DataFrame con los postes filtrados y eliminar duplicados
postes_filtrados = pd.DataFrame(postes_cercanos).drop_duplicates()

# Filtrar el DataFrame `postes` basado en las rutas de los postes filtrados
if not postes_filtrados.empty:
    rutas_filtradas = postes_filtrados['Ruta'].unique()
    postes = postes[postes['Ruta'].isin(rutas_filtradas)]
else:
    print("No se encontraron postes dentro de los radios definidos.")


#######################################################################################################################################################
############################ Buscamos los postes mas cercanos
s = time.time()

    # Definir la función de cálculo de distancias utilizando `haversine`
def calculate_distances_vectorized(point_coords, postes):
    # Extraer coordenadas de los postes
    postes_coords = postes[['Lat', 'Long']].to_numpy()

    # Calcular las distancias utilizando la fórmula de haversine en un bucle vectorizado
    distances = np.array([
        haversine(point_coords[0], point_coords[1], lat, lon) 
        for lat, lon in postes_coords
    ])

    return distances

trayectory_data = []
found_posts = 0  # Contador de postes encontrados
last_position_index = 0  # Índice de la última posición procesada

# Iterar sobre el DataFrame df
while last_position_index < len(df):
    row = df.iloc[last_position_index]
    punto_coords = (row['GPS (Lat.) [deg]'], row['GPS (Long.) [deg]'])

    # Calcular todas las distancias vectorizadas
    distances = calculate_distances_vectorized(punto_coords, postes)

    # Encontrar el índice del poste más cercano dentro del rango de 20 metros
    closest_poste_index = np.argmin(distances)
    distancia_minima = distances[closest_poste_index]

    if distancia_minima < 20:  # Verificar si hay un poste cercano
        poste_cercano = postes.iloc[closest_poste_index]
        trayectory_data.append({
            'GPS (Lat.) [deg]': row['GPS (Lat.) [deg]'],
            'GPS (Long.) [deg]': row['GPS (Long.) [deg]'],
            'Ruta': poste_cercano['Ruta'],
            'KM': poste_cercano['KM'],
            'Distancia': distancia_minima
        })

        found_posts += 1

        # Avanzar a la siguiente fila que acumule al menos 15 metros
        accumulated_distance = 0
        while accumulated_distance < 20:
            last_position_index += 1

            if last_position_index >= len(df):
                #print("Final del DataFrame alcanzado. Saliendo del bucle principal.")
                break

            accumulated_distance += df.iloc[last_position_index]['Distancia']

    else:
        # Si no se encuentra un poste cercano, avanzar directamente a la siguiente fila
        accumulated_distance = 0
        while accumulated_distance < 20:
            last_position_index += 1
            
            if last_position_index >= len(df):
                #print("Final del DataFrame alcanzado. Saliendo del bucle principal.")
                break

            accumulated_distance += df.iloc[last_position_index]['Distancia']

    # Verificar si se alcanzó el final y no se puede avanzar más
    if last_position_index >= len(df):
        #print("Final del DataFrame alcanzado. Saliendo del bucle principal.")
        break

# Crear el DataFrame final
trayectory = pd.DataFrame(trayectory_data)
print(trayectory)

f = time.time()
tiempo_total_minutos = 0
duracion_minutos = (f - s) / 60
tiempo_total_minutos += duracion_minutos
print("")
print(f"{duracion_minutos:.2f} minutos.")

    GPS (Lat.) [deg]  GPS (Long.) [deg]   Ruta    KM  Distancia
0         -34.734521         -55.975465    8.0  29.0  17.087678
1         -34.734645         -55.975627    8.0  29.0  17.116866
2         -34.740890         -55.983924    8.0  28.0  15.687447
3         -34.752220         -56.001043    8.0  26.0  17.304281
4         -34.757600         -56.009132    8.0  25.0  17.247301
5         -34.757716         -56.009302    8.0  25.0  15.905759
6         -34.755488         -56.019025   74.0  25.0  12.106601
7         -34.755302         -56.019063   74.0  25.0   9.655764
8         -34.747069         -56.022721   74.0  26.0  11.664610
9         -34.746907         -56.022848   74.0  26.0  13.888135
10        -34.739306         -56.028111   74.0  27.0  11.562004
11        -34.739145         -56.028216   74.0  27.0  10.631647
12        -34.739248         -56.028176   74.0  27.0   6.990891
13        -34.746985         -56.022779   74.0  26.0   6.454981
14        -34.755337         -56.019032 

### **Motivación del Código**

El objetivo de este bloque de código es limpiar el DataFrame de trayectoria (`trayectory`) eliminando posibles falsos positivos y garantizando que los postes asociados realmente pertenezcan a la ruta válida del trayecto. Esto es fundamental para evitar errores como asociar postes de rutas perpendiculares o no relacionadas. El proceso se realiza en dos etapas: primero, eliminamos duplicados por ruta y kilómetro, quedándonos con la fila más cercana dentro de un grupo; segundo, identificamos y descartamos cambios de ruta inconsistentes, asegurándonos de mantener solo secuencias válidas en términos de continuidad.

---

### **Explicación del Código**

#### **1. Eliminación de duplicados en ruta y kilómetro**

En este paso, se asegura que no existan múltiples filas en la trayectoria asociadas al mismo poste (definido por `Ruta` y `KM`):
- **Agrupación por Ruta y KM**:
  - Se recorren las filas del DataFrame `trayectory`, agrupando filas consecutivas que tienen el mismo `Ruta` y `KM`.
  - Esto garantiza que todas las filas que se refieren al mismo poste sean consideradas juntas.
- **Selección de la fila con menor distancia**:
  - Dentro de cada grupo, se elige la fila con la menor distancia al poste.
  - Esto es importante porque puede haber varias lecturas cercanas al mismo poste, pero solo la más precisa (es decir, la de menor distancia) es relevante.

El resultado de este paso es un DataFrame intermedio llamado `trayectory_cleaned`, donde cada poste aparece como máximo una vez.

---

#### **2. Filtrado de falsos positivos**

En esta segunda etapa, se verifica que los cambios de ruta sean válidos y que no se incluyan postes de rutas no relacionadas:
- **Validación de secuencias de ruta**:
  - Las filas del DataFrame `trayectory_cleaned` se recorren una a una. Si la ruta cambia, el código evalúa si este cambio está respaldado por una secuencia mínima de filas consecutivas en la nueva ruta.
  - El umbral definido por `MIN_CONSECUTIVE_ROWS` (en este caso, 2) garantiza que un cambio de ruta se considere válido solo si hay al menos dos lecturas consecutivas en la nueva ruta.
- **Almacenamiento de rutas válidas**:
  - Si el cambio de ruta no cumple con este umbral, las filas correspondientes son descartadas.
  - Esto asegura que los postes de rutas perpendiculares o no relacionadas no se incluyan como parte de la trayectoria final.

El resultado es un DataFrame final llamado `trayectory_final`, donde solo se conservan las filas que cumplen con los criterios de proximidad y continuidad de ruta.

---

### **Razón para la Estrategia de Limpieza**

- **Evitar ruido en los datos**: 
  - La inclusión de postes no relacionados puede introducir ruido significativo, afectando cualquier análisis posterior. Este código garantiza que cada fila en el DataFrame final esté asociada a una ruta válida.
- **Optimización de resultados**:
  - Al eliminar duplicados y filas inconsistentes, se mejora la precisión del análisis sin necesidad de procesar datos redundantes.
- **Control sobre la continuidad**:
  - La verificación de secuencias consecutivas permite detectar y filtrar casos en los que las lecturas salten a rutas no relacionadas debido a errores en los datos o trayectorias complejas.

---

### **Conclusión**

Este bloque de código aplica un proceso exhaustivo de limpieza al DataFrame de trayectoria. Combina técnicas para eliminar duplicados y validar cambios de ruta, garantizando que el conjunto final de datos sea consistente y preciso. Esto es esencial para reducir errores y mejorar la confiabilidad del análisis geográfico.


In [25]:
#######################################################################################################################################################
############################ Eliminamos los acercamientos a los postes

filtered_rows = []

i = 0
while i < len(trayectory):
    current_group = [trayectory.iloc[i]]
    
    # Comparar la fila actual con las siguientes
    while (
        i + 1 < len(trayectory) and
        trayectory.iloc[i]['Ruta'] == trayectory.iloc[i + 1]['Ruta'] and
        trayectory.iloc[i]['KM'] == trayectory.iloc[i + 1]['KM']
    ):
        current_group.append(trayectory.iloc[i + 1])
        i += 1
    
    # Quedarse con la fila de menor distancia dentro del grupo
    min_distance_row = min(current_group, key=lambda x: x['Distancia'])
    filtered_rows.append(min_distance_row)
    
    i += 1

trayectory_cleaned = pd.DataFrame(filtered_rows)



#######################################################################################################################################################
############################ Filtramos posibles falsos positivos de postes 

filtered_rows = []  # Lista para almacenar las filas filtradas
temp_rows = []      # Lista temporal para almacenar filas de una posible nueva ruta
current_route = None  # Ruta actual que se considera válida

# Umbral para considerar un cambio de ruta como válido
MIN_CONSECUTIVE_ROWS = 2

for index, row in trayectory_cleaned.iterrows():
    ruta_actual = row['Ruta']
    km_actual = row['KM']
    
    if current_route is None:
        # Si es la primera fila, inicializar la ruta actual y agregar la fila a `filtered_rows`
        current_route = ruta_actual
        filtered_rows.append(row)
        continue
    
    if ruta_actual == current_route:
        # Si la ruta es igual a la ruta válida actual, agregar la fila a `filtered_rows`
        filtered_rows.append(row)
        # Si hay filas temporales, descartarlas porque volvimos a la ruta válida
        if temp_rows:
            temp_rows = []
    else:
        # Si la ruta cambia, guardar la fila en `temp_rows`
        temp_rows.append(row)
        
        # Evaluar si hay suficientes filas consecutivas en la nueva ruta
        if len(temp_rows) >= MIN_CONSECUTIVE_ROWS:
            # Considerar el cambio de ruta como válido
            current_route = ruta_actual
            filtered_rows.extend(temp_rows)
            temp_rows = []

# Crear un nuevo DataFrame a partir de las filas filtradas
trayectory_final = pd.DataFrame(filtered_rows)

trayectory_final

,GPS (Lat.) [deg],GPS (Long.) [deg],Ruta,KM,Distancia
0,-34.734521,-55.975465,8.0,29.0,17.087678
2,-34.740890,-55.983924,8.0,28.0,15.687447
3,-34.752220,-56.001043,8.0,26.0,17.304281
5,-34.757716,-56.009302,8.0,25.0,15.905759
7,-34.755302,-56.019063,74.0,25.0,9.655764
8,-34.747069,-56.022721,74.0,26.0,11.664610
12,-34.739248,-56.028176,74.0,27.0,6.990891
13,-34.746985,-56.022779,74.0,26.0,6.454981
14,-34.755337,-56.019032,74.0,25.0,7.058652
17,-34.763551,-56.017828,8.0,24.0,13.673618


### **Motivación del Código**

El propósito de este bloque es identificar los carteles que están asociados a la trayectoria procesada y validada en pasos anteriores. Para optimizar el proceso, se trabaja en dos etapas: primero, se filtran los radares (un subconjunto más pequeño del DataFrame de carteles) que están asociados a la trayectoria; luego, se filtran los carteles usando esta selección previa, calculando las distancias a la trayectoria para determinar cuáles están efectivamente vinculados y en qué punto exacto de la trayectoria aparecen.

---

### **Explicación del Código**

#### **1. Filtrar los radares asociados a la trayectoria**

- **Rutas únicas de postes**: 
  - Se obtiene el conjunto de rutas únicas de los postes asociados a la trayectoria. Esto reduce el espacio de búsqueda inicial y garantiza que solo se consideren rutas relevantes.
- **Filtrado de radares**:
  - Se filtran los radares cuya ruta coincide con las rutas presentes en los postes. Esto reduce significativamente el tamaño del DataFrame inicial (`radares`) que debe procesarse.
- **Cálculo de distancias vectorizadas**:
  - Para cada radar filtrado, se calcula su distancia a todos los puntos de la trayectoria (`df`). Usar un enfoque vectorizado (con NumPy y `haversine`) permite calcular todas las distancias de forma eficiente.
- **Selección del radar más cercano**:
  - Se identifica el punto de la trayectoria más cercano al radar dentro de un radio de 20 metros. Si existe un punto dentro del rango, se almacena el radar y su índice correspondiente en un diccionario (`radares_encontrados`). Esto asegura que solo se procesen radares efectivamente asociados a la trayectoria.

---

#### **2. Filtrar carteles asociados a los radares**

- **Subconjunto inicial de carteles**:
  - Se filtra el DataFrame `carteles` utilizando los radares identificados en el paso anterior. Esto reduce aún más el espacio de búsqueda, ya que solo se consideran carteles vinculados a radares relevantes.
- **Cálculo de distancias**:
  - Para cada cartel filtrado, se calcula su distancia a todos los puntos de la trayectoria. Este cálculo, también vectorizado con NumPy y `haversine`, asegura que el proceso sea rápido incluso con conjuntos de datos grandes.
- **Identificación del punto más cercano**:
  - Se determina el índice del punto de la trayectoria más cercano a cada cartel, junto con su distancia. Esto permite asociar cada cartel con un punto específico de la trayectoria.
- **Almacenamiento y ordenamiento**:
  - Los índices y distancias calculados se almacenan en columnas del DataFrame `carteles_filtrado`. Luego, el DataFrame se ordena por el índice más cercano para facilitar el análisis posterior.

---

### **Optimización del Proceso**

- **Reducción del espacio de búsqueda**:
  - Filtrar primero por rutas y luego por radares asegura que solo se procesen carteles potencialmente relevantes. Esto reduce el tiempo de cómputo de manera significativa.
- **Uso de cálculos vectorizados**:
  - En ambas etapas, los cálculos de distancia se realizan de forma vectorizada utilizando NumPy. Este enfoque es mucho más eficiente que iterar sobre las filas con bucles, especialmente para grandes volúmenes de datos.
- **Asociación directa de índices**:
  - Al asociar cada cartel con un índice específico de la trayectoria, se simplifica cualquier análisis posterior que dependa de esta relación.

---

### **Conclusión**

Este bloque de código es esencial para identificar los carteles asociados a la trayectoria, pero lo hace de manera eficiente al reducir progresivamente el espacio de búsqueda. Los cálculos vectorizados y el uso de estructuras como diccionarios para almacenar resultados intermedios garantizan un buen rendimiento, incluso con conjuntos de datos grandes. El resultado final es un DataFrame ordenado de carteles asociados a puntos específicos de la trayectoria, listo para análisis o procesamiento posterior.


In [26]:
#######################################################################################################################################################
############################ Buscamos que RADARES estan en el trayecto 
s = time.time()

radares_encontrados = {}

# Filtrar radares por las rutas presentes en `postes`
rutas_unicas_postes = postes['Ruta'].unique()
radares_filtrados = radares[radares['Ruta'].isin(rutas_unicas_postes)]

# Convertir las coordenadas de `df` a arrays numpy para cálculos rápidos
df_coords = df[['GPS (Lat.) [deg]', 'GPS (Long.) [deg]']].to_numpy()

# Iterar sobre los radares filtrados
for _, row_r in radares_filtrados.iterrows():
    radar_coords = np.array([row_r['Lat'], row_r['Long']])
    radar_codigo = row_r['Código Radar']
    
    # Calcular las distancias de manera vectorizada
    distances = haversine(
        radar_coords[0], radar_coords[1],
        df_coords[:, 0], df_coords[:, 1]
    )
    
    # Encontrar el índice más cercano dentro del rango de 20 metros
    within_range = np.where(distances < 20)[0]
    if within_range.size > 0:
        nearest_index = within_range[0]  # Obtener el primer índice dentro del rango
        radares_encontrados[radar_codigo] = nearest_index



#######################################################################################################################################################
############################ Obtengo el sub dataframe que contiene al a los carteles asosciados a dichos radares

# Filtrar el DataFrame `carteles` usando los nombres de los radares del diccionario `radares_encontrados`
nombres_radares = list(radares_encontrados.keys())
carteles_filtrado = carteles[carteles['Código Radar'].isin(nombres_radares)].copy()

# Convertir las coordenadas del DataFrame `df` a un array de numpy para cálculos rápidos
df_coords = df[['GPS (Lat.) [deg]', 'GPS (Long.) [deg]']].to_numpy()

# Inicializar las listas para guardar los resultados
indices_cercanos = []
distancias_minimas = []

# Recorrer las filas de `carteles_filtrado` para calcular distancias
carteles_coords = carteles_filtrado[['Lat', 'Long']].to_numpy()

for cartel_coord in carteles_coords:
    # Calcular las distancias de manera vectorizada usando haversine
    distances = np.array([
        haversine(cartel_coord[0], cartel_coord[1], df_coord[0], df_coord[1])
        for df_coord in df_coords
    ])

    # Encontrar el índice del punto más cercano y la distancia mínima
    min_index = np.argmin(distances)
    min_distance = distances[min_index]

    indices_cercanos.append(min_index)
    distancias_minimas.append(min_distance)

# Agregar las columnas calculadas al DataFrame
carteles_filtrado['índice'] = indices_cercanos
carteles_filtrado['distancia'] = distancias_minimas

# Ordenar el DataFrame por la columna 'índice'
carteles_filtrado = carteles_filtrado.sort_values(by='índice', ascending=True)

f = time.time()
tiempo_total_minutos = 0
duracion_minutos = (f - s) / 60
tiempo_total_minutos += duracion_minutos
print(f"carteles asosciados a dichos radares    {duracion_minutos:.2f} minutos.")


carteles asosciados a dichos radares    0.03 minutos.


In [27]:
radares_encontrados

{'M015': 11776,
 'M017': 4753,
 'M084 - E': 19092,
 'M084 - S': 18974,
 'M110': 7113}

In [28]:
carteles_filtrado

,Nombre de video,Lat,Long,Código Radar,Unnamed: 4,índice,distancia
43,GX010022,-34.735524,-55.976790,M017,NaN,1808,0.000000
44,GX010022,-34.756300,-56.007209,M017,NaN,4618,0.000000
229,GX010022,-34.757496,-56.018624,M110,NaN,6868,0.000000
230,GX010022,-34.750942,-56.019940,M110,NaN,9958,0.000000
41,GX010020,-34.767620,-56.023243,M015,NaN,11917,15.578302
176,GX010022,-34.792831,-56.080291,M084 - S,NaN,17958,0.000000
177,GX010022,-34.793313,-56.093784,M084 - S,NaN,18768,0.000000
175,GX010014,-34.791724,-56.101697,M084 - E,NaN,19301,9.043151


### **Motivación del Código**

Una vez identificada la trayectoria de circulación, las rutas, los postes kilométricos asociados, los radares y los carteles, es esencial determinar el **sentido de circulación** del trayecto. Esto es crucial porque algunos radares y carteles pueden estar asociados a ambos sentidos de circulación en una misma ruta y para este proyecto debemos de diferenciarlos. Para resolver esto, primero identificamos el sentido de desplazamiento según los postes kilométricos cercanos, basándonos en si nos acercamos o alejamos de un poste específico. Posteriormente, filtramos los carteles para quedarnos únicamente con aquellos que corresponden al sentido de circulación correcto, utilizando reglas de codificación previamente definidas.

---

### **Explicación del Código**

#### **1. Determinar el sentido de desplazamiento**

- **Creación de la columna 'sentido'**:
  - Se añade una columna al DataFrame `carteles_filtrado` para almacenar el sentido de circulación ('+' o '-') de cada cartel.
  
- **Casos especiales: trayectoria con un único punto**:
  - Si la trayectoria tiene solo un punto, se calcula el sentido comparando la distancia del cartel a un poste anterior y al punto de trayectoria. Esto asegura que se pueda determinar el sentido incluso en trayectorias mínimas.

- **Iteración sobre pares de puntos consecutivos**:
  - Para trayectorias con múltiples puntos, se evalúan pares consecutivos de puntos en `trayectory_final`.
  - **Distancias inicial y final**:
    - Se calcula la distancia del cartel al punto inicial y al punto final del par. 
    - La suma de estas distancias se utiliza para determinar qué par de puntos está más cercano al cartel.
  - **Identificación del sentido**:
    - Comparando los valores de `KM` de los puntos inicial y final, se determina el sentido de desplazamiento ('+' si el trayecto avanza en dirección creciente, '-' en caso contrario).

#### **2. Filtrar carteles según el sentido de circulación**

- **Aplicación de reglas de codificación**:
  - Se filtran los carteles utilizando reglas definidas por el equipo de transporte:
    - Si el cartel está en el sentido '+' y contiene códigos como `-B` o `-E`, se descarta.
    - Si está en el sentido '-' y contiene códigos como `-S` o `-A`, también se descarta.
  - Esto asegura que solo se retengan los carteles relevantes para el sentido de circulación real del trayecto.

---

### **Optimización y Uso del Código**

- **Procesamiento eficiente**:
  - Se minimizan cálculos redundantes al considerar únicamente pares consecutivos de puntos y limitar el análisis a carteles previamente filtrados por trayectoria y radares.
- **Reglas claras**:
  - Las reglas de codificación permiten una clasificación directa y eliminan posibles falsos positivos asociados a carteles que no pertenecen al trayecto real.

---

### **Conclusión**

Este bloque es fundamental para garantizar que los carteles y radares procesados estén asociados al trayecto correcto y al sentido real de circulación. La combinación de cálculos basados en distancia y reglas de codificación asegura precisión en los resultados. El resultado final es un conjunto de carteles correctamente clasificados por sentido, listo para análisis o procesamiento posterior.


In [10]:
trayectory_final

,GPS (Lat.) [deg],GPS (Long.) [deg],Ruta,KM,Distancia
1,-34.830778,-56.010124,200.0,21.0,5.321733
3,-34.826453,-56.000275,200.0,22.0,4.947196
6,-34.822444,-55.991769,200.0,23.0,3.550398
7,-34.826375,-56.000305,200.0,22.0,13.300359


In [11]:
carteles_filtrado

,Nombre de video,Lat,Long,Código Radar,Unnamed: 4,índice,distancia
0,GX010024,-34.829707,-56.007576,M001 - A,NaN,140,7.057004
1,GX010024,-34.828071,-56.003778,M001 - A,NaN,383,9.065465
2,GX020025,-34.824717,-55.996601,M001 - B,NaN,2096,3.686963


In [29]:
#######################################################################################################################################################
############################ Determino el sentido de desplazamiento
s =time.time()
# Crear o inicializar la columna 'sentido' en el DataFrame `carteles_filtrados`
carteles_filtrado['sentido'] = None

# Recorrer cada fila de `carteles_filtrados`
for index_c, row_c in carteles_filtrado.iterrows():
    # Obtener el punto geográfico actual
    punto_cartel = (row_c['Lat'], row_c['Long'])
    
    # Variable para almacenar la suma mínima y los índices de los postes que la generaron
    suma_minima = float('inf')
    poste_inicial = None
    poste_final = None

    # Manejar el caso en que `trayectory_final` tenga solo una fila
    if len(trayectory_final) == 1:
        # Obtener el único punto de `trayectory_final`
        punto_unico = (trayectory_final.iloc[0]['GPS (Lat.) [deg]'], trayectory_final.iloc[0]['GPS (Long.) [deg]'])
        km_unico = trayectory_final.iloc[0]['KM']
        
        # Buscar el poste anterior en el DataFrame `postes`
        postes2 = pd.read_excel("Postes_KM_2025.xlsx")
        poste_anterior = postes2[postes2['KM'] < km_unico].sort_values(by='KM', ascending=False).head(1)
        
        if poste_anterior.empty:
            print(f"Advertencia: No se encontró un poste anterior para el índice {index_c}.")
            carteles_filtrado.at[index_c, 'sentido'] = None
            continue

        punto_anterior = (poste_anterior.iloc[0]['Lat'], poste_anterior.iloc[0]['Long'])
        
        # Calcular las distancias al punto inicial y final
        distancia_inicial = geodesic(punto_anterior, punto_unico).meters
        distancia_final = geodesic(punto_cartel, punto_unico).meters
        
        # Determinar el sentido
        if distancia_inicial < distancia_final:
            carteles_filtrado.at[index_c, 'sentido'] = '+'
        else:
            carteles_filtrado.at[index_c, 'sentido'] = '-'
        continue

    # Iterar sobre pares consecutivos de puntos en `trayectory_final`
    for i in range(len(trayectory_final) - 1):
        # Obtener las coordenadas de los postes consecutivos
        punto_inicial = (trayectory_final.iloc[i]['GPS (Lat.) [deg]'], trayectory_final.iloc[i]['GPS (Long.) [deg]'])
        punto_final = (trayectory_final.iloc[i + 1]['GPS (Lat.) [deg]'], trayectory_final.iloc[i + 1]['GPS (Long.) [deg]'])
        
        # Calcular las distancias desde el punto actual de `carteles_filtrados`
        distancia_inicial = geodesic(punto_cartel, punto_inicial).meters
        distancia_final = geodesic(punto_cartel, punto_final).meters
        
        # Calcular la suma de las distancias
        suma_distancias = distancia_inicial + distancia_final
        
        # Si la suma actual es menor que la suma mínima, actualizar
        if suma_distancias < suma_minima:
            suma_minima = suma_distancias
            poste_inicial = i
            poste_final = i + 1
    ################################################################################  Debreriamos evaluar las dos distancias minimas y en caso de que la resta de esta sea menor a x (20metros) la agregamos a df otra muestra del mismo cartel y me quedaria uno con + y otro con -, posteriormente se descarta solo el que no deberia estar
    # Identificar el sentido según los valores de KM
    km_inicial = trayectory_final.iloc[poste_inicial]['KM']
    km_final = trayectory_final.iloc[poste_final]['KM']
    
    if km_inicial < km_final:
        carteles_filtrado.at[index_c, 'sentido'] = '+'
    else:
        carteles_filtrado.at[index_c, 'sentido'] = '-'


f = time.time()
tiempo_total_minutos = 0
duracion_minutos = (f - s) / 60
tiempo_total_minutos += duracion_minutos
print(f"sentido de trayectoria    {duracion_minutos:.2f} minutos.")

#######################################################################################################################################################
############################ Nos quedamos solo con los carteles que estan en el mismo setido de circulacion

carteles_filtrado = carteles_filtrado[
    ~((carteles_filtrado['sentido'] == '-') & (carteles_filtrado['Código Radar'].str.contains('-S|-A', regex=True))) |
    ((carteles_filtrado['sentido'] == '+') & (carteles_filtrado['Código Radar'].str.contains('-B|-E', regex=True)))
]

carteles_filtrado

sentido de trayectoria    0.00 minutos.


,Nombre de video,Lat,Long,Código Radar,Unnamed: 4,índice,distancia,sentido
43,GX010022,-34.735524,-55.976790,M017,NaN,1808,0.000000,-
44,GX010022,-34.756300,-56.007209,M017,NaN,4618,0.000000,-
229,GX010022,-34.757496,-56.018624,M110,NaN,6868,0.000000,-
230,GX010022,-34.750942,-56.019940,M110,NaN,9958,0.000000,+
41,GX010020,-34.767620,-56.023243,M015,NaN,11917,15.578302,-
176,GX010022,-34.792831,-56.080291,M084 - S,NaN,17958,0.000000,+
177,GX010022,-34.793313,-56.093784,M084 - S,NaN,18768,0.000000,+
175,GX010014,-34.791724,-56.101697,M084 - E,NaN,19301,9.043151,+


In [30]:
#######################################################################################################################################################
############################ Generar clips y frames
s = time.time()
# Ruta base donde se encuentran los videos (mismo directorio que el script, en la carpeta "dev")
base_video_path = os.getcwd()

# Ruta base para las carpetas que se crearán
output_base_path = os.path.join(base_video_path, "dev", "output2")

# Diccionario para contar las subcarpetas por "Código Radar"
carpetas_creadas = {}

# Ubicar el video correspondiente (asumiendo que es único para todo el DataFrame)
video_name = f"{carteles_filtrado.iloc[0]['Nombre de video']}.mp4"
video_path = os.path.join(base_video_path, "dev", video_name)

# Verificar si el video existe
if not os.path.exists(video_path):
    print(f"El video {video_name} no existe en {base_video_path}. Abortando...")
else:
    # Abrir el video una sola vez
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"No se pudo abrir el video {video_name}. Abortando...")
    else:
        # Obtener propiedades del video
        fourcc = cv2.VideoWriter_fourcc(*"H264")
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Procesar cada fila del DataFrame
        for index, row in carteles_filtrado.iterrows():
            # Obtener el nombre de la carpeta principal por "Código Radar"
            folder_name = row['Código Radar']
            folder_path = os.path.join(output_base_path, folder_name)

            # Inicializar el contador de subcarpetas si es la primera vez que encontramos este "Código Radar"
            if folder_name not in carpetas_creadas:
                carpetas_creadas[folder_name] = 0

            # Incrementar el contador y definir el nombre de la subcarpeta
            carpetas_creadas[folder_name] += 1
            subfolder_name = f"cartel {carpetas_creadas[folder_name]}"

            # Crear las subcarpetas para frames y clips
            subfolder_frames_path = os.path.join(folder_path, "frames", subfolder_name)
            subfolder_clips_path = os.path.join(folder_path, "clips", subfolder_name)
            os.makedirs(subfolder_frames_path, exist_ok=True)
            os.makedirs(subfolder_clips_path, exist_ok=True)

            # Obtener el índice y buscar el valor 'cts' en el DataFrame 'df'
            indice = row['índice']
            cts_value = df.loc[indice, 'cts']  # Acceder al valor en el DataFrame df
            center_msec = cts_value  # Mantener los milisegundos directamente

            # Calcular el tiempo de inicio y fin del clip (en milisegundos)
            clip_start_msec = max(0, center_msec - 5000)  # 5 segundos antes del centro
            clip_end_msec = center_msec + 5000  # 5 segundos después del centro

            # Configurar el códec para el clip
            clip_path = os.path.join(subfolder_clips_path, f"{folder_name}_clip.mp4")
            out = cv2.VideoWriter(clip_path, fourcc, fps, (frame_width, frame_height))

            # Posicionarse en el tiempo de inicio del clip
            cap.set(cv2.CAP_PROP_POS_MSEC, clip_start_msec)

            # Leer y escribir frames hasta el tiempo de finalización
            while cap.get(cv2.CAP_PROP_POS_MSEC) <= clip_end_msec:
                ret, frame = cap.read()
                if not ret:
                    break

                # Agregar texto al frame
                minutes = int(center_msec // 60000)
                seconds = int((center_msec % 60000) // 1000)
                text = f"{video_name}, Start: {minutes:02d}:{seconds:02d}"
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 1.5
                font_color = (0, 0, 255)  # Rojo
                thickness = 2
                text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
                text_x = frame_width - text_size[0] - 15
                text_y = 40
                cv2.putText(frame, text, (text_x, text_y), font, font_scale, font_color, thickness)

                out.write(frame)

            # Liberar el objeto VideoWriter para este clip
            out.release()

            # Calcular los tiempos para los 10 frames equiespaciados entre 2.5 y 7.5 segundos
            start_frame = int(2.5 * fps)
            end_frame = int(7.5 * fps)
            step = (end_frame - start_frame) // 5
            frames_to_extract = [start_frame + i * step for i in range(5)]

            # Extraer los frames
            for frame_number in frames_to_extract:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
                ret, frame = cap.read()
                if ret:
                    # Agregar texto al frame
                    minutes = int(center_msec // 60000)
                    seconds = int((center_msec % 60000) // 1000)
                    text = f"{video_name}, Start: {minutes:02d}:{seconds:02d}"
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    font_scale = 1.5
                    font_color = (0, 0, 255)  # Rojo
                    thickness = 2
                    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
                    text_x = frame_width - text_size[0] - 15
                    text_y = 40
                    cv2.putText(frame, text, (text_x, text_y), font, font_scale, font_color, thickness)

                    # Guardar el frame
                    frame_name = f"frame_{frame_number}.jpg"
                    frame_path = os.path.join(subfolder_frames_path, frame_name)
                    cv2.imwrite(frame_path, frame)

        # Liberar el objeto VideoCapture
        cap.release()

f = time.time()
tiempo_total_minutos = (f - s) / 60
print(f"clips y frames generados en {tiempo_total_minutos:.2f} minutos.")

clips y frames generados en 3.60 minutos.


## Codigo base 

In [31]:
import tkinter as tk
from tkinter import filedialog, font
from tkinter import ttk, filedialog, font
import os
import pandas as pd
from geopy.distance import geodesic
import cv2
import numpy as np
import time
from haversine import haversine, Unit
from math import radians, sin, cos, sqrt, atan2
from threading import Thread

##########################################################################################################################################
########################### Seleccion de directorios 
##########################################################################################################################################

def seleccionar_directorio():
    root = tk.Tk()
    root.title("Sign Scan")  # Título de la ventana
    root.geometry("700x300")  # Tamaño inicial de la ventana
    root.minsize(500, 250)  # Tamaño mínimo de la ventana

    italic_font = font.Font(family="Helvetica", size=12, slant="italic")

    titulo = tk.Label(root, text="Sign Scan", font=("Helvetica", 36, "bold"))
    titulo.pack(pady=15)

    descripcion = tk.Label(
        root,
        text="Seleccione la carpeta donde se encuentran los videos del relevamiento\n"
             "y los CSV correspondientes a la trayectoria.\n"
             "Recuerde que dicha carpeta debe contener una subcarpeta llamada 'master'\n"
             "con la información de la línea base.",
        font=italic_font,
        justify="center",
    )
    descripcion.pack(pady=10)

    # Ajustar el ancho del texto cuando se redimensione la ventana
    def ajustar_ancho(event):
        wrap_width = int(root.winfo_width() * 0.9)  # Ajustar al 90% del ancho de la ventana
        descripcion.config(wraplength=wrap_width)

    root.bind("<Configure>", ajustar_ancho)  # Vincular evento de redimensionado

    # Variables para almacenar el directorio
    base = None
    master_directory = None

    # Función que se ejecuta al presionar el botón
    def seleccionar_carpeta():
        nonlocal base, master_directory
        base = filedialog.askdirectory(title="Selecciona la carpeta base")
        if base:
            print(f"Directorio base seleccionado: {base}")
            master_directory = os.path.join(base, "master")
            print(f"Directorio 'master': {master_directory}")
        else:
            print("No se seleccionó ningún directorio.")
        root.destroy()  

    boton = tk.Button(root, text="Seleccionar directorio", command=seleccionar_carpeta, font=("Helvetica", 10))
    boton.pack(pady=20)

    root.mainloop()

    return base, master_directory


def iniciar_interfaz_progreso(total_docs):
    """
    Inicia la interfaz con barra de progreso e historial para el proceso.
    
    Args:
        total_docs (int): Número total de documentos a procesar.
    """
    def cerrar_interfaz():
        root.destroy()

    global progreso_bar, historial, root
    root = tk.Tk()
    root.title("Progreso de Procesamiento")
    root.geometry("600x400")

    # Etiqueta del título
    titulo = tk.Label(root, text="Procesando Archivos", font=("Helvetica", 16, "bold"))
    titulo.pack(pady=10)

    # Barra de progreso
    progreso_bar = ttk.Progressbar(root, orient="horizontal", length=400, mode="determinate")
    progreso_bar.pack(pady=10)
    progreso_bar['maximum'] = total_docs  # Tamaño total de la barra
    progreso_bar['value'] = 0  # Inicia en 0

    # Historial de archivos procesados
    historial = tk.Text(root, wrap=tk.WORD, height=15, width=70, state="normal")
    historial.pack(pady=10)

    # Botón para cerrar la ventana al final del proceso
    cerrar_btn = tk.Button(root, text="Cerrar", font=("Helvetica", 12), command=cerrar_interfaz)
    cerrar_btn.pack(pady=10)

    # Mensaje inicial en el historial
    historial.insert(tk.END, f"Archivos a procesar: {total_docs}\n")
    historial.insert(tk.END, "\n", "normal")

    # Mantener la ventana abierta
    root.update()

def actualizar_interfaz_progreso(current_doc, docs_processed, puntos_encontrados):
    """
    Actualiza la interfaz con el progreso actual, mostrando el documento procesado y los puntos encontrados.

    Args:
        current_doc (str): Nombre del documento que se acaba de procesar.
        docs_processed (int): Cantidad de documentos procesados.
        puntos_encontrados (list): Lista de puntos encontrados en el documento.
    """
    # Actualizar la barra de progreso
    progreso_bar['value'] = docs_processed
    progreso_bar.update()

    # Añadir información al historial
    historial.insert(tk.END, f"Procesado: {current_doc}\n")
    historial.insert(tk.END, f"Puntos encontrados: {len(puntos_encontrados)}\n")
    
    # Listar los puntos encontrados
    for punto in puntos_encontrados:
        historial.insert(tk.END, f" - {punto}\n")
    
    historial.insert(tk.END, "\n", "normal")
    
    # Mantener el scroll al final
    historial.yview(tk.END)
    root.update()


def finalizar_interfaz_progreso():
    """
    Finaliza el proceso principal, destruye la ventana principal y crea una nueva ventana independiente para mostrar el historial.
    """
    # Guardar el contenido del historial
    contenido_historial = historial.get(1.0, tk.END)

    # Destruir la ventana principal
    root.destroy()

    # Crear una nueva ventana para la notificación final
    ventana_final = tk.Tk()
    ventana_final.title("Proceso Finalizado")
    ventana_final.geometry("600x400")

    # Mensaje de procesamiento finalizado
    etiqueta_final = tk.Label(ventana_final, text="¡Procesamiento completado!", font=("Helvetica", 14, "bold"))
    etiqueta_final.pack(pady=10)

    # Frame para el historial con scroll
    frame_historial = tk.Frame(ventana_final)
    frame_historial.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    # Scrollbar
    scrollbar = tk.Scrollbar(frame_historial)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    # Text widget para mostrar el historial
    historial_final = tk.Text(frame_historial, wrap=tk.WORD, yscrollcommand=scrollbar.set, font=("Helvetica", 10))
    historial_final.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

    # Configurar el scrollbar para que controle el texto del historial
    scrollbar.config(command=historial_final.yview)

    # Insertar el contenido del historial
    historial_final.insert(tk.END, contenido_historial)
    historial_final.configure(state="disabled")  # Hacer el historial de solo lectura

    # Botón para cerrar la ventana final
    boton_cerrar = tk.Button(ventana_final, text="Cerrar", command=ventana_final.destroy, font=("Helvetica", 12), pady=5)
    boton_cerrar.pack(pady=10)

    # Mantener la nueva ventana abierta
    ventana_final.mainloop()



# Direcoritos principales
base, master_directory = seleccionar_directorio()
carteles_path = os.path.join(master_directory, "radares_geolocalizacion.xlsx")
radares_path = os.path.join(master_directory, "radares_geolocalizacion.xlsx")
postes_path = os.path.join(master_directory, "Postes_KM_2025.xlsx")


#Archivos base 
carteles = pd.read_excel(carteles_path, sheet_name="Linea Base")
carteles['Código Radar'] = carteles['Código Radar'].str.replace(' ', '', regex=True)

radares = pd.read_excel(radares_path, sheet_name="Radares")
radares['Código Radar'] = radares['Código Radar'].str.replace(' ', '', regex=True)

postes = pd.read_excel(postes_path)
filtros = radares['Ruta'].tolist()
postes = postes[postes['Ruta'].isin(filtros)]


# Función de haversine vectorizada
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radio de la Tierra en metros
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2.0) ** 2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

##########################################################################################################################################
##########################################################################################################################################
########################### Proceso general
##########################################################################################################################################
##########################################################################################################################################

# Obtener todos los archivos CSV en el directorio base
csv_files = [f for f in os.listdir(base) if f.endswith('.csv')]

total_docs = len(csv_files)
iniciar_interfaz_progreso(total_docs)

docs_processed = 0

for csv_file in csv_files:
    file_path = os.path.join(base, csv_file)
    df = pd.read_csv(file_path)

    df['Distancia'] = df.apply(
        lambda row: haversine(
            row['GPS (Lat.) [deg]'], row['GPS (Long.) [deg]'], 
            df.iloc[row.name - 1]['GPS (Lat.) [deg]'], df.iloc[row.name - 1]['GPS (Long.) [deg]']
        ) if row.name > 0 else 0,  # La primera fila no tiene punto anterior
        axis=1
    )

    postes = pd.read_excel(postes_path)
    filtros = radares['Ruta'].tolist()
    postes = postes[postes['Ruta'].isin(filtros)]
    
    print(file_path)

    # Cambio el nombre de video al pre fijo de el csv de lectrua
    prefix = csv_file.split("_")[0]
    carteles['Nombre de video'] = prefix

    #######################################################################################################################################################
    ############################ Buscamos los postes mas cercanos

        # Definir la función de cálculo de distancias utilizando haversine
    def calculate_distances_vectorized(point_coords, postes):
        # Extraer coordenadas de los postes
        postes_coords = postes[['Lat', 'Long']].to_numpy()

        # Calcular las distancias utilizando la fórmula de haversine en un bucle vectorizado
        distances = np.array([
            haversine(point_coords[0], point_coords[1], lat, lon) 
            for lat, lon in postes_coords
        ])

        return distances

    trayectory_data = []
    found_posts = 0  # Contador de postes encontrados
    last_position_index = 0  # Índice de la última posición procesada

    # Iterar sobre el DataFrame df
    while last_position_index < len(df):
        row = df.iloc[last_position_index]
        punto_coords = (row['GPS (Lat.) [deg]'], row['GPS (Long.) [deg]'])

        # Calcular todas las distancias vectorizadas
        distances = calculate_distances_vectorized(punto_coords, postes)

        # Encontrar el índice del poste más cercano dentro del rango de 20 metros
        closest_poste_index = np.argmin(distances)
        distancia_minima = distances[closest_poste_index]

        if distancia_minima < 20:  # Verificar si hay un poste cercano
            poste_cercano = postes.iloc[closest_poste_index]
            trayectory_data.append({
                'GPS (Lat.) [deg]': row['GPS (Lat.) [deg]'],
                'GPS (Long.) [deg]': row['GPS (Long.) [deg]'],
                'Ruta': poste_cercano['Ruta'],
                'KM': poste_cercano['KM'],
                'Distancia': distancia_minima
            })

            found_posts += 1

            # Avanzar a la siguiente fila que acumule al menos 15 metros
            accumulated_distance = 0
            while accumulated_distance < 20:
                last_position_index += 1

                if last_position_index >= len(df):
                    #print("Final del DataFrame alcanzado. Saliendo del bucle principal.")
                    break

                accumulated_distance += df.iloc[last_position_index]['Distancia']

        else:
            # Si no se encuentra un poste cercano, avanzar directamente a la siguiente fila
            accumulated_distance = 0
            while accumulated_distance < 20:
                last_position_index += 1
                
                if last_position_index >= len(df):
                    #print("Final del DataFrame alcanzado. Saliendo del bucle principal.")
                    break

                accumulated_distance += df.iloc[last_position_index]['Distancia']

        # Verificar si se alcanzó el final y no se puede avanzar más
        if last_position_index >= len(df):
            #print("Final del DataFrame alcanzado. Saliendo del bucle principal.")
            break

    # Crear el DataFrame final
    trayectory = pd.DataFrame(trayectory_data)


    #######################################################################################################################################################
    ############################ Eliminamos los acercamientos a los postes

    filtered_rows = []

    i = 0
    while i < len(trayectory):
        current_group = [trayectory.iloc[i]]
        
        # Comparar la fila actual con las siguientes
        while (
            i + 1 < len(trayectory) and
            trayectory.iloc[i]['Ruta'] == trayectory.iloc[i + 1]['Ruta'] and
            trayectory.iloc[i]['KM'] == trayectory.iloc[i + 1]['KM']
        ):
            current_group.append(trayectory.iloc[i + 1])
            i += 1
        
        # Quedarse con la fila de menor distancia dentro del grupo
        min_distance_row = min(current_group, key=lambda x: x['Distancia'])
        filtered_rows.append(min_distance_row)
        
        i += 1

    trayectory_cleaned = pd.DataFrame(filtered_rows)

    #######################################################################################################################################################
    ############################ Filtramos posibles falsos positivos de postes 

    filtered_rows = []  # Lista para almacenar las filas filtradas
    temp_rows = []      # Lista temporal para almacenar filas de una posible nueva ruta
    current_route = None  # Ruta actual que se considera válida

    # Umbral para considerar un cambio de ruta como válido
    MIN_CONSECUTIVE_ROWS = 2

    for index, row in trayectory_cleaned.iterrows():
        ruta_actual = row['Ruta']
        km_actual = row['KM']
        
        if current_route is None:
            # Si es la primera fila, inicializar la ruta actual y agregar la fila a `filtered_rows`
            current_route = ruta_actual
            filtered_rows.append(row)
            continue
        
        if ruta_actual == current_route:
            # Si la ruta es igual a la ruta válida actual, agregar la fila a `filtered_rows`
            filtered_rows.append(row)
            # Si hay filas temporales, descartarlas porque volvimos a la ruta válida
            if temp_rows:
                temp_rows = []
        else:
            # Si la ruta cambia, guardar la fila en `temp_rows`
            temp_rows.append(row)
            
            # Evaluar si hay suficientes filas consecutivas en la nueva ruta
            if len(temp_rows) >= MIN_CONSECUTIVE_ROWS:
                # Considerar el cambio de ruta como válido
                current_route = ruta_actual
                filtered_rows.extend(temp_rows)
                temp_rows = []

    # Crear un nuevo DataFrame a partir de las filas filtradas
    trayectory_final = pd.DataFrame(filtered_rows)



    #######################################################################################################################################################
    ############################ Buscamos que RADARES estan en el trayecto 
    s = time.time()

    radares_encontrados = {}

    # Filtrar radares por las rutas presentes en `postes`
    rutas_unicas_postes = postes['Ruta'].unique()
    radares_filtrados = radares[radares['Ruta'].isin(rutas_unicas_postes)]

    # Convertir las coordenadas de `df` a arrays numpy para cálculos rápidos
    df_coords = df[['GPS (Lat.) [deg]', 'GPS (Long.) [deg]']].to_numpy()

    # Iterar sobre los radares filtrados
    for _, row_r in radares_filtrados.iterrows():
        radar_coords = np.array([row_r['Lat'], row_r['Long']])
        radar_codigo = row_r['Código Radar']
        
        # Calcular las distancias de manera vectorizada
        distances = haversine(
            radar_coords[0], radar_coords[1],
            df_coords[:, 0], df_coords[:, 1]
        )
        
        # Encontrar el índice más cercano dentro del rango de 20 metros
        within_range = np.where(distances < 20)[0]
        if within_range.size > 0:
            nearest_index = within_range[0]  # Obtener el primer índice dentro del rango
            radares_encontrados[radar_codigo] = nearest_index

    radares_encontrados

    
    f = time.time()
    tiempo_total_minutos = 0
    duracion_minutos = (f - s) / 60
    tiempo_total_minutos += duracion_minutos
    print(f"RADARES que estan en el trayecto    {duracion_minutos:.2f} minutos.")

    #######################################################################################################################################################
    ############################ Obtengo el sub dataframe que contiene al a los carteles asosciados a dichos radares
    s = time.time()

    # Filtrar el DataFrame `carteles` usando los nombres de los radares del diccionario `radares_encontrados`
    nombres_radares = list(radares_encontrados.keys())
    carteles_filtrado = carteles[carteles['Código Radar'].isin(nombres_radares)].copy()

    # Convertir las coordenadas del DataFrame `df` a un array de numpy para cálculos rápidos
    df_coords = df[['GPS (Lat.) [deg]', 'GPS (Long.) [deg]']].to_numpy()

    # Inicializar las listas para guardar los resultados
    indices_cercanos = []
    distancias_minimas = []

    # Recorrer las filas de `carteles_filtrado` para calcular distancias
    carteles_coords = carteles_filtrado[['Lat', 'Long']].to_numpy()

    for cartel_coord in carteles_coords:
        # Calcular las distancias de manera vectorizada usando haversine
        distances = np.array([
            haversine(cartel_coord[0], cartel_coord[1], df_coord[0], df_coord[1])
            for df_coord in df_coords
        ])

        # Encontrar el índice del punto más cercano y la distancia mínima
        min_index = np.argmin(distances)
        min_distance = distances[min_index]

        indices_cercanos.append(min_index)
        distancias_minimas.append(min_distance)

    # Agregar las columnas calculadas al DataFrame
    carteles_filtrado['índice'] = indices_cercanos
    carteles_filtrado['distancia'] = distancias_minimas

    # Ordenar el DataFrame por la columna 'índice'
    carteles_filtrado = carteles_filtrado.sort_values(by='índice', ascending=True)

    f = time.time()
    tiempo_total_minutos = 0
    duracion_minutos = (f - s) / 60
    tiempo_total_minutos += duracion_minutos
    print(f"carteles asosciados a dichos radares    {duracion_minutos:.2f} minutos.")

    #######################################################################################################################################################
    ############################ Determino el sentido de desplazamiento
    s =time.time()
    # Crear o inicializar la columna 'sentido' en el DataFrame `carteles_filtrados`
    carteles_filtrado['sentido'] = None

    # Recorrer cada fila de `carteles_filtrados`
    for index_c, row_c in carteles_filtrado.iterrows():
        # Obtener el punto geográfico actual
        punto_cartel = (row_c['Lat'], row_c['Long'])
        
        # Variable para almacenar la suma mínima y los índices de los postes que la generaron
        suma_minima = float('inf')
        poste_inicial = None
        poste_final = None

        # Manejar el caso en que `trayectory_final` tenga solo una fila
        if len(trayectory_final) == 1:
            # Obtener el único punto de `trayectory_final`
            punto_unico = (trayectory_final.iloc[0]['GPS (Lat.) [deg]'], trayectory_final.iloc[0]['GPS (Long.) [deg]'])
            km_unico = trayectory_final.iloc[0]['KM']
            
            # Buscar el poste anterior en el DataFrame `postes`
            postes2 = pd.read_excel(postes_path)
            poste_anterior = postes2[postes2['KM'] < km_unico].sort_values(by='KM', ascending=False).head(1)
            
            if poste_anterior.empty:
                print(f"Advertencia: No se encontró un poste anterior para el índice {index_c}.")
                carteles_filtrado.at[index_c, 'sentido'] = None
                continue

            punto_anterior = (poste_anterior.iloc[0]['Lat'], poste_anterior.iloc[0]['Long'])
            
            # Calcular las distancias al punto inicial y final
            distancia_inicial = geodesic(punto_anterior, punto_unico).meters
            distancia_final = geodesic(punto_cartel, punto_unico).meters
            
            # Determinar el sentido
            if distancia_inicial < distancia_final:
                carteles_filtrado.at[index_c, 'sentido'] = '+'
            else:
                carteles_filtrado.at[index_c, 'sentido'] = '-'
            continue

        # Iterar sobre pares consecutivos de puntos en `trayectory_final`
        for i in range(len(trayectory_final) - 1):
            # Obtener las coordenadas de los postes consecutivos
            punto_inicial = (trayectory_final.iloc[i]['GPS (Lat.) [deg]'], trayectory_final.iloc[i]['GPS (Long.) [deg]'])
            punto_final = (trayectory_final.iloc[i + 1]['GPS (Lat.) [deg]'], trayectory_final.iloc[i + 1]['GPS (Long.) [deg]'])
            
            # Calcular las distancias desde el punto actual de `carteles_filtrados`
            distancia_inicial = geodesic(punto_cartel, punto_inicial).meters
            distancia_final = geodesic(punto_cartel, punto_final).meters
            
            # Calcular la suma de las distancias
            suma_distancias = distancia_inicial + distancia_final
            
            # Si la suma actual es menor que la suma mínima, actualizar
            if suma_distancias < suma_minima:
                suma_minima = suma_distancias
                poste_inicial = i
                poste_final = i + 1

        # Identificar el sentido según los valores de KM
        km_inicial = trayectory_final.iloc[poste_inicial]['KM']
        km_final = trayectory_final.iloc[poste_final]['KM']
        
        if km_inicial < km_final:
            carteles_filtrado.at[index_c, 'sentido'] = '+'
        else:
            carteles_filtrado.at[index_c, 'sentido'] = '-'


    f = time.time()
    tiempo_total_minutos = 0
    duracion_minutos = (f - s) / 60
    tiempo_total_minutos += duracion_minutos
    print(f"sentido de trayectoria    {duracion_minutos:.2f} minutos.")

    #######################################################################################################################################################
    ############################ Nos quedamos solo con los carteles que estan en el mismo setido de circulacion

    carteles_filtrado = carteles_filtrado[
        ~((carteles_filtrado['sentido'] == '-') & (carteles_filtrado['Código Radar'].str.contains('-S|-A', regex=True))) |
        ((carteles_filtrado['sentido'] == '+') & (carteles_filtrado['Código Radar'].str.contains('-B|-E', regex=True)))
    ]


    #######################################################################################################################################################
    ############################ Generar clips y frames
    s = time.time()
    # Ruta base donde se encuentran los videos (mismo directorio que el script, en la carpeta "dev")
    base_video_path = base

    # Ruta base para las carpetas que se crearán
    output_base_path = os.path.join(base_video_path, "OUTPUT")

    # Diccionario para contar las subcarpetas por "Código Radar"
    carpetas_creadas = {}

    video_name =  f"{carteles_filtrado.iloc[0]['Nombre de video']}.mp4"
    video_path = os.path.join(base_video_path, video_name)

    # Verificar si el video existe
    if not os.path.exists(video_path):
        print(f"El video {video_name} no existe en {base_video_path}. Abortando...")
    else:
        # Abrir el video una sola vez
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"No se pudo abrir el video {video_name}. Abortando...")
        else:
            # Obtener propiedades del video
            fourcc = cv2.VideoWriter_fourcc(*"H264")
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = cap.get(cv2.CAP_PROP_FPS)

            # Procesar cada fila del DataFrame
            for index, row in carteles_filtrado.iterrows():
                # Obtener el nombre de la carpeta principal por "Código Radar"
                folder_name = row['Código Radar']
                folder_path = os.path.join(output_base_path, folder_name)

                # Inicializar el contador de subcarpetas si es la primera vez que encontramos este "Código Radar"
                if folder_name not in carpetas_creadas:
                    carpetas_creadas[folder_name] = 0

                # Incrementar el contador y definir el nombre de la subcarpeta
                carpetas_creadas[folder_name] += 1
                subfolder_name = f"cartel {carpetas_creadas[folder_name]}"

                # Crear las subcarpetas para frames y clips
                subfolder_frames_path = os.path.join(folder_path, "frames", subfolder_name)
                subfolder_clips_path = os.path.join(folder_path, "clips", subfolder_name)
                os.makedirs(subfolder_frames_path, exist_ok=True)
                os.makedirs(subfolder_clips_path, exist_ok=True)

                # Obtener el índice y buscar el valor 'cts' en el DataFrame 'df'
                indice = row['índice']
                cts_value = df.loc[indice, 'cts']  # Acceder al valor en el DataFrame df
                center_msec = cts_value  # Mantener los milisegundos directamente

                # Calcular el tiempo de inicio y fin del clip (en milisegundos)
                clip_start_msec = max(0, center_msec - 5000)  # 5 segundos antes del centro
                clip_end_msec = center_msec + 5000  # 5 segundos después del centro

                # Configurar el códec para el clip
                clip_path = os.path.join(subfolder_clips_path, f"{folder_name}_clip.mp4")
                out = cv2.VideoWriter(clip_path, fourcc, fps, (frame_width, frame_height))

                # Posicionarse en el tiempo de inicio del clip
                cap.set(cv2.CAP_PROP_POS_MSEC, clip_start_msec)

                # Leer y escribir frames hasta el tiempo de finalización
                while cap.get(cv2.CAP_PROP_POS_MSEC) <= clip_end_msec:
                    ret, frame = cap.read()
                    if not ret:
                        break

                    # Agregar texto al frame
                    minutes = int(center_msec // 60000)
                    seconds = int((center_msec % 60000) // 1000)
                    text = f"{video_name}, Start: {minutes:02d}:{seconds:02d}"
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    font_scale = 1.5
                    font_color = (0, 0, 255)  # Rojo
                    thickness = 2
                    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
                    text_x = frame_width - text_size[0] - 15
                    text_y = 40
                    cv2.putText(frame, text, (text_x, text_y), font, font_scale, font_color, thickness)

                    out.write(frame)

                # Liberar el objeto VideoWriter para este clip
                out.release()

                # Procesar el clip generado para extraer frames
                clip_cap = cv2.VideoCapture(clip_path)
                if not clip_cap.isOpened():
                    print(f"No se pudo abrir el clip generado {clip_path}. Saltando...")
                    continue

                # Calcular los tiempos para los 5 frames equiespaciados entre 2.5 y 7.5 segundos
                clip_fps = clip_cap.get(cv2.CAP_PROP_FPS)
                start_frame = int(2.5 * clip_fps)
                end_frame = int(7.5 * clip_fps)
                step = (end_frame - start_frame) // 5
                frames_to_extract = [start_frame + i * step for i in range(5)]

                # Extraer los frames del clip
                for frame_number in frames_to_extract:
                    clip_cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
                    ret, frame = clip_cap.read()
                    if ret:
                        # Agregar texto al frame
                        minutes = int(center_msec // 60000)
                        seconds = int((center_msec % 60000) // 1000)
                        text = f"{video_name}, Start: {minutes:02d}:{seconds:02d}"
                        font = cv2.FONT_HERSHEY_SIMPLEX
                        font_scale = 1.5
                        font_color = (0, 0, 255)  # Rojo
                        thickness = 2
                        text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
                        text_x = frame_width - text_size[0] - 15
                        text_y = 40
                        cv2.putText(frame, text, (text_x, text_y), font, font_scale, font_color, thickness)

                        # Guardar el frame
                        frame_name = f"frame_{frame_number}.jpg"
                        frame_path = os.path.join(subfolder_frames_path, frame_name)
                        cv2.imwrite(frame_path, frame)

                # Liberar el objeto VideoCapture del clip
                clip_cap.release()

            # Liberar el objeto VideoCapture
            cap.release()

    f = time.time()
    tiempo_total_minutos = (f - s) / 60
    print(f"clips y frames generados en {tiempo_total_minutos:.2f} minutos.")

    puntos_encontrados = carteles_filtrado['Código Radar'].drop_duplicates().tolist()
    docs_processed += 1
    actualizar_interfaz_progreso(csv_file, docs_processed, puntos_encontrados)


finalizar_interfaz_progreso()






Directorio base seleccionado: C:/Users/fpino/OneDrive - CSI Ingenieros - CIEMSA/Escritorio/dev
Directorio 'master': C:/Users/fpino/OneDrive - CSI Ingenieros - CIEMSA/Escritorio/dev\master
C:/Users/fpino/OneDrive - CSI Ingenieros - CIEMSA/Escritorio/dev\GX010259_HERO11 Black-GPS9.csv
RADARES que estan en el trayecto    0.00 minutos.
carteles asosciados a dichos radares    0.00 minutos.
sentido de trayectoria    0.00 minutos.
clips y frames generados en 1.52 minutos.


# Aplicación 2 

In [32]:
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import folium
from datetime import datetime
from urllib.parse import quote
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

class ImageEvaluatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Evaluador de Imágenes")
        self.root.geometry("1200x800")
        self.root.minsize(800, 600)

        # Variables
        self.current_image_index = 0
        self.current_folder_index = 0
        self.image_paths = []
        self.image_titles = []
        self.current_subfolder_images = []
        self.evaluations = []
        self.directory = None

        # Vincular teclas para navegación
        self.root.bind("<Left>", lambda e: self.prev_image())
        self.root.bind("<Right>", lambda e: self.next_image())

        # Crear frames
        self.frame_top = tk.Frame(self.root, bg="lightblue", height=50)
        self.frame_middle = tk.Frame(self.root, bg="gray")
        self.frame_bottom = tk.Frame(self.root, bg="white", height=144)

        self.frame_top.pack(side=tk.TOP, fill=tk.X, expand=False)
        self.frame_middle.pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        self.frame_bottom.pack(side=tk.TOP, fill=tk.X, expand=False)

        # Widgets
        self.label_title = tk.Label(self.frame_top, text="Evaluador de Imágenes", font=("Arial", 18), bg="lightblue")
        self.label_title.grid(row=0, column=0, padx=10, sticky="w")

        self.btn_select_dir = tk.Button(self.frame_top, text="Seleccionar Directorio", command=self.select_directory, font=("Arial", 12))
        self.btn_select_dir.grid(row=0, column=1, padx=10, sticky="e")

        self.label_image_title = tk.Label(self.frame_top, text="", font=("Arial", 14), bg="lightblue")
        self.label_image_title.grid(row=1, column=0, columnspan=2, pady=5)

        self.frame_top.grid_columnconfigure(0, weight=1)
        self.frame_top.grid_columnconfigure(1, weight=0)

        self.canvas = tk.Canvas(self.frame_middle, bg="black")
        self.canvas.pack(fill=tk.BOTH, expand=True)

        eval_frame = tk.Frame(self.frame_bottom)
        eval_frame.pack(pady=10)

        tk.Label(eval_frame, text="Visibilidad (1-10):", font=("Arial", 14)).grid(row=0, column=0, padx=5)
        self.entry_visibilidad = tk.Entry(eval_frame, width=5, font=("Arial", 14))
        self.entry_visibilidad.grid(row=0, column=1, padx=5)

        tk.Label(eval_frame, text="Estado (1-10):", font=("Arial", 14)).grid(row=0, column=2, padx=5)
        self.entry_estado = tk.Entry(eval_frame, width=5, font=("Arial", 14))
        self.entry_estado.grid(row=0, column=3, padx=5)

        tk.Label(eval_frame, text="Vandalismo (1-10):", font=("Arial", 14)).grid(row=0, column=4, padx=5)
        self.entry_vandalismo = tk.Entry(eval_frame, width=5, font=("Arial", 14))
        self.entry_vandalismo.grid(row=0, column=5, padx=5)

        tk.Label(eval_frame, text="Comentarios:", font=("Arial", 14)).grid(row=1, column=0, padx=5, sticky="w")
        self.entry_comentarios = tk.Entry(eval_frame, width=50, font=("Arial", 14))
        self.entry_comentarios.grid(row=1, column=1, columnspan=5, padx=5, pady=5)

        button_frame = tk.Frame(self.frame_bottom)
        button_frame.pack(pady=10)

        self.btn_prev_image = tk.Button(button_frame, text="Anterior Imagen", command=self.prev_image, font=("Arial", 12))
        self.btn_prev_image.grid(row=0, column=0, padx=10)

        self.btn_save_next = tk.Button(button_frame, text="Guardar y Siguiente Carpeta", command=self.save_and_next_folder, font=("Arial", 12))
        self.btn_save_next.grid(row=0, column=1, padx=10)

        self.btn_export = tk.Button(button_frame, text="Exportar Datos y Mapa", command=self.export_data_and_map, font=("Arial", 12))
        self.btn_export.grid(row=0, column=2, padx=10)

        self.btn_next_image = tk.Button(button_frame, text="Siguiente Imagen", command=self.next_image, font=("Arial", 12))
        self.btn_next_image.grid(row=0, column=3, padx=10)

    def select_directory(self):
        directory = filedialog.askdirectory(title="Selecciona un directorio")
        if directory:
            self.directory = directory
            self.load_folders(directory)
        else:
            messagebox.showinfo("Información", "No seleccionaste ningún directorio.")

    def load_folders(self, directory):
        self.image_paths.clear()
        self.image_titles.clear()
        self.current_folder_index = 0

        output_dir = os.path.join(directory, "OUTPUT")
        if not os.path.exists(output_dir):
            messagebox.showerror("Error", "No se encontró la carpeta OUTPUT en el directorio seleccionado.")
            return

        for folder in os.listdir(output_dir):
            folder_path = os.path.join(output_dir, folder)
            frames_path = os.path.join(folder_path, "frames")

            if os.path.isdir(frames_path):
                for subfolder in os.listdir(frames_path):
                    subfolder_path = os.path.join(frames_path, subfolder)
                    if os.path.isdir(subfolder_path):
                        images = [
                            os.path.join(subfolder_path, img)
                            for img in os.listdir(subfolder_path)
                            if img.lower().endswith((".jpg", ".png", ".jpeg"))
                        ]
                        if images:
                            self.image_paths.append(images)
                            self.image_titles.append(f"{folder}/{subfolder}")

        if self.image_paths:
            self.current_image_index = 0
            self.load_subfolder_images(0)
            self.btn_save_next.config(state=tk.NORMAL)
        else:
            messagebox.showinfo("Información", "No se encontraron imágenes en el directorio especificado.")

    def load_subfolder_images(self, folder_index):
        if 0 <= folder_index < len(self.image_paths):
            self.current_subfolder_images = self.image_paths[folder_index]
            self.current_image_index = 0
            self.show_image()

    def show_image(self):
        if 0 <= self.current_image_index < len(self.current_subfolder_images):
            image_path = self.current_subfolder_images[self.current_image_index]
            image_title = self.image_titles[self.current_folder_index]

            image = Image.open(image_path)
            image.thumbnail((1500, 750))
            image_tk = ImageTk.PhotoImage(image)

            self.canvas.delete("all")
            self.canvas.create_image(self.canvas.winfo_width() // 2, self.canvas.winfo_height() // 2, image=image_tk, anchor="center")
            self.canvas.image = image_tk

            self.label_image_title.config(text=f"{image_title} - Imagen {self.current_image_index + 1} de {len(self.current_subfolder_images)}")

    def prev_image(self):
        if self.current_image_index > 0:
            self.current_image_index -= 1
            self.show_image()

    def next_image(self):
        if self.current_image_index < len(self.current_subfolder_images) - 1:
            self.current_image_index += 1
            self.show_image()

    def save_and_next_folder(self):
        visibilidad = self.entry_visibilidad.get()
        estado = self.entry_estado.get()
        vandalismo = self.entry_vandalismo.get()
        comentarios = self.entry_comentarios.get()

        if not (visibilidad.isdigit() and estado.isdigit() and vandalismo.isdigit()):
            messagebox.showerror("Error", "Las calificaciones deben ser números entre 1 y 10.")
            return

        current_path = self.current_subfolder_images[self.current_image_index]
        self.evaluations.append({
            "Carpeta": self.image_titles[self.current_folder_index],
            "Imagen Seleccionada": os.path.basename(current_path),
            "Ruta": current_path,
            "Visibilidad": int(visibilidad),
            "Estado": int(estado),
            "Vandalismo": int(vandalismo),
            "Comentarios": comentarios
        })

        if self.current_folder_index < len(self.image_paths) - 1:
            self.current_folder_index += 1
            self.load_subfolder_images(self.current_folder_index)
        else:
            self.btn_save_next.config(state=tk.DISABLED)
            messagebox.showinfo("Finalizado", "Has evaluado todas las carpetas.")

    def export_data_and_map(self):
        if not self.evaluations:
            messagebox.showerror("Error", "No hay evaluaciones para exportar.")
            return

        df = pd.DataFrame(self.evaluations)

        # Convertir rutas a formato file://
        df["Ruta"] = df["Ruta"].apply(lambda x: f"file:///{quote(x.replace('\\', '/'))}")

        # Cargar datos de carteles
        carteles_path = os.path.join(self.directory, "master", "radares_geolocalizacion.xlsx")
        if not os.path.exists(carteles_path):
            messagebox.showerror("Error", "No se encontró el archivo radares_geolocalizacion.xlsx.")
            return

        carteles = pd.read_excel(carteles_path, sheet_name="Linea Base")
        carteles["Código Radar"] = carteles["Código Radar"].str.replace(" ", "")

        df["Carpeta Base"] = df["Carpeta"].str.split("/").str[0]
        nombres_unicos = df["Carpeta Base"].unique()
        carteles_filtrados = carteles[carteles["Código Radar"].isin(nombres_unicos)].copy()

        contador_carteles = {}
        numero_cartel = []

        for codigo in carteles_filtrados["Código Radar"]:
            if codigo not in contador_carteles:
                contador_carteles[codigo] = 1
            else:
                contador_carteles[codigo] += 1
            numero_cartel.append(f"cartel {contador_carteles[codigo]}")

        carteles_filtrados["numero cartel"] = numero_cartel
        carteles_filtrados["Radar/Cartel"] = carteles_filtrados["Código Radar"] + "/" + carteles_filtrados["numero cartel"]

        carteles_combined = pd.merge(
            carteles_filtrados,
            df,
            left_on="Radar/Cartel",
            right_on="Carpeta",
            how="inner"
        )

        if "Lat" not in carteles_combined.columns or "Long" not in carteles_combined.columns:
            messagebox.showerror("Error", "El DataFrame combinado no contiene las columnas 'Lat' y 'Long'.")
            return

        # Cargar datos de radares
        radares_path = os.path.join(self.directory, "master", "radares_geolocalizacion.xlsx")
        radares = pd.read_excel(radares_path, sheet_name="Radares")
        radares["Código Radar"] = radares["Código Radar"].str.replace(" ", "")

        # Filtrar radares por los códigos existentes en carteles_combined
        radares_filtrados = radares[radares["Código Radar"].isin(carteles_combined["Código Radar"])]

        # Crear colores únicos para cada Código Radar
        folium_colors = [
            'red', 'blue', 'green', 'purple', 'orange', 'darkred',
            'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
            'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
            'gray', 'black', 'lightgray'
        ]
        color_map = {
            code: folium_colors[i % len(folium_colors)]
            for i, code in enumerate(carteles_combined["Código Radar"].unique())
        }
        carteles_combined["color"] = carteles_combined["Código Radar"].map(color_map)

        # Crear mapa
        m = folium.Map(location=[carteles_combined["Lat"].mean(), carteles_combined["Long"].mean()], zoom_start=10)

        # Agregar puntos y líneas al mapa
        for codigo, group in carteles_combined.groupby("Código Radar"):
            coords = []
            for _, row in group.iterrows():
                coords.append((row["Lat"], row["Long"]))
                folium.Marker(
                    location=(row["Lat"], row["Long"]),
                    popup=folium.Popup(
                        f"""
                        <b>{row['Carpeta']}</b><br>
                        Visibilidad: {row['Visibilidad']}<br>
                        Estado: {row['Estado']}<br>
                        Vandalismo: {row['Vandalismo']}<br>
                        Comentarios: {row['Comentarios']}<br>
                        <img src="{row['Ruta']}" width="600" height="350">
                        """,
                        max_width=600,
                    ),
                    icon=folium.Icon(color=row["color"]),
                ).add_to(m)

            # Agregar punto del radar y conectar con línea
            radar_data = radares_filtrados[radares_filtrados["Código Radar"] == codigo]
            for _, radar in radar_data.iterrows():
                radar_coords = (radar["Lat"], radar["Long"])
                coords.append(radar_coords)
                folium.Circle(
                    location=radar_coords,
                    radius=30,
                    color=color_map[codigo],
                    fill=True,
                    fill_color=color_map[codigo],
                ).add_to(m)

            if len(coords) > 1:
                folium.PolyLine(coords, color=color_map[codigo], weight=2.5).add_to(m)
        
        # Agregar funcionalidad para postes kilometricos######################################################################################
        # Función de haversine vectorizada
        def haversine(lat1, lon1, lat2, lon2):
            R = 6371000  # Radio de la Tierra en metros
            phi1 = np.radians(lat1)
            phi2 = np.radians(lat2)
            delta_phi = np.radians(lat2 - lat1)
            delta_lambda = np.radians(lon2 - lon1)

            a = np.sin(delta_phi / 2.0) ** 2 + \
                np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
            return R * c

        postes_path = os.path.join(self.directory, "master", "Postes_KM_2025.xlsx")
        if not os.path.exists(postes_path):
            messagebox.showerror("Error", "No se encontró el archivo Postes_KM_2025.xlsx.")
            return

        postes = pd.read_excel(postes_path)

        # Filtrar postes por rutas presentes en radares_filtrados
        postes_filtrados = postes[postes["Ruta"].isin(radares_filtrados["Ruta"])].copy()

        # Filtrar postes por proximidad a los radares
        postes_finales = []
        for _, radar in radares_filtrados.iterrows():
            lat_radar, long_radar = radar["Lat"], radar["Long"]
            postes_filtrados["distancia"] = haversine(
                postes_filtrados["Lat"].values, postes_filtrados["Long"].values,
                lat_radar, long_radar
            )
            postes_proximos = postes_filtrados[postes_filtrados["distancia"] <= 1050]
            postes_finales.append(postes_proximos)

        # Combinar los postes finales
        if postes_finales:
            postes_finales = pd.concat(postes_finales).drop_duplicates()
        else:
            postes_finales = pd.DataFrame()  # Si no hay postes cercanos

        # Agregar los postes al mapa como círculos clickeables
        for _, poste in postes_finales.iterrows():
            folium.CircleMarker(
                location=(poste["Lat"], poste["Long"]),
                radius=7,  # Tamaño ajustable del círculo
                color="black",
                fill=True,
                fill_color="gray",
                fill_opacity = 0.99,
                popup=folium.Popup(
                    f"""
                    <b>Ruta:</b> {poste['Ruta']}<br>
                    <b>KM:</b> {poste['KM']}<br>
                    """,
                    max_width=300,
                )
            ).add_to(m)

        # Guardar en el directorio seleccionado
        if self.directory:
            date_str = datetime.now().strftime("%Y-%m-%d")
            xlsx_path = os.path.join(self.directory, f"sign_scan_{date_str}.xlsx")
            html_path = os.path.join(self.directory, f"sign_scan_{date_str}.html")

            # Seleccionar columnas específicas
            export_columns = [
                "Ruta",
                "Código Radar",
                "numero cartel",
                "Imagen Seleccionada",
                "Lat",
                "Long",
                "Visibilidad",
                "Estado",
                "Vandalismo",
                "Comentarios"
            ]
            export_df = carteles_combined[export_columns]

            try:
                export_df.to_excel(xlsx_path, index=False, engine='openpyxl')
                # Cargar el archivo Excel exportado
                wb = load_workbook(xlsx_path)
                ws = wb.active

                # Definir colores para el formato condicional
                red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")  # Rojo
                yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # Amarillo
                green_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Verde

                # Determinar el índice de la columna "Estado"
                header = [cell.value for cell in ws[1]]
                estado_col_index = header.index("Estado") + 1  # Sumar 1 porque las columnas de openpyxl empiezan en 1

                # Aplicar formato condicional fila por fila
                for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=estado_col_index, max_col=estado_col_index):
                    for cell in row:
                        if cell.value is not None:
                            if 1 <= cell.value <= 4:
                                # Pintar toda la fila de rojo
                                for cell_in_row in ws[cell.row]:
                                    cell_in_row.fill = red_fill
                            elif 5 <= cell.value <= 8:
                                # Pintar toda la fila de amarillo
                                for cell_in_row in ws[cell.row]:
                                    cell_in_row.fill = yellow_fill
                            elif 9 <= cell.value <= 10:
                                # Pintar toda la fila de verde
                                for cell_in_row in ws[cell.row]:
                                    cell_in_row.fill = green_fill

                # Guardar cambios
                wb.save(xlsx_path)
                m.save(html_path)
                messagebox.showinfo("Exportado", f"Datos exportados a:\n{xlsx_path}\nMapa guardado en:\n{html_path}")
            except PermissionError:
                messagebox.showerror("Error", "No se pudo guardar el archivo. Verifica que no esté en uso o protegido.")
        else:
            messagebox.showerror("Error", "No se seleccionó un directorio para exportar.")


if __name__ == "__main__":
    root = tk.Tk()
    app = ImageEvaluatorApp(root)
    root.mainloop()


agregar mapa y listo

## Con puntitos

In [26]:
class ImageEvaluatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Evaluador de Imágenes")
        self.root.geometry("1200x800")
        self.root.minsize(800, 600)

        # Variables
        self.current_image_index = 0
        self.current_folder_index = 0
        self.image_paths = []
        self.image_titles = []
        self.current_subfolder_images = []
        self.evaluations = []

        # Variables para las calificaciones
        self.visibilidad_var = tk.IntVar(value=1)
        self.estado_var = tk.IntVar(value=1)
        self.vandalismo_var = tk.IntVar(value=1)

        # Vincular teclas para navegación
        self.root.bind("<Left>", lambda e: self.prev_image())
        self.root.bind("<Right>", lambda e: self.next_image())

        # Crear frames para dividir la pantalla
        self.frame_top = tk.Frame(self.root, bg="lightblue", height=50)
        self.frame_middle = tk.Frame(self.root, bg="gray")
        self.frame_bottom = tk.Frame(self.root, bg="white", height=144)

        # Configurar layout basado en proporciones
        self.frame_top.pack(side=tk.TOP, fill=tk.X, expand=False)
        self.frame_middle.pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        self.frame_bottom.pack(side=tk.TOP, fill=tk.X, expand=False)

        # Widgets en frame_top (Título, botón y título de la imagen)
        self.label_title = tk.Label(self.frame_top, text="Evaluador de Imágenes", font=("Arial", 18), bg="lightblue")
        self.label_title.grid(row=0, column=0, padx=10, sticky="w")

        self.btn_select_dir = tk.Button(
            self.frame_top, text="Seleccionar Directorio", command=self.select_directory, font=("Arial", 12)
        )
        self.btn_select_dir.grid(row=0, column=1, padx=10, sticky="e")

        self.label_image_title = tk.Label(self.frame_top, text="", font=("Arial", 14), bg="lightblue")
        self.label_image_title.grid(row=1, column=0, columnspan=2, pady=5)

        self.frame_top.grid_columnconfigure(0, weight=1)
        self.frame_top.grid_columnconfigure(1, weight=0)

        # Canvas para la imagen en frame_middle
        self.canvas = tk.Canvas(self.frame_middle, bg="black")
        self.canvas.pack(fill=tk.BOTH, expand=True)

        # Widgets en frame_bottom (Evaluación y botones)
        eval_frame = tk.Frame(self.frame_bottom)
        eval_frame.pack(pady=10)

        # Grupo de botones de radio para Visibilidad
        tk.Label(eval_frame, text="Visibilidad (1-5):", font=("Arial", 14)).grid(row=0, column=0, padx=10)
        for i in range(1, 6):
            tk.Radiobutton(eval_frame, text=str(i), variable=self.visibilidad_var, value=i, font=("Arial", 12)).grid(
                row=0, column=i, padx=5
            )

        # Grupo de botones de radio para Estado
        tk.Label(eval_frame, text="Estado (1-5):", font=("Arial", 14)).grid(row=1, column=0, padx=10)
        for i in range(1, 6):
            tk.Radiobutton(eval_frame, text=str(i), variable=self.estado_var, value=i, font=("Arial", 12)).grid(
                row=1, column=i, padx=5
            )

        # Grupo de botones de radio para Vandalismo
        tk.Label(eval_frame, text="Vandalismo (1-5):", font=("Arial", 14)).grid(row=2, column=0, padx=10)
        for i in range(1, 6):
            tk.Radiobutton(eval_frame, text=str(i), variable=self.vandalismo_var, value=i, font=("Arial", 12)).grid(
                row=2, column=i, padx=5
            )

        # Campo de comentarios
        tk.Label(eval_frame, text="Comentarios:", font=("Arial", 14)).grid(row=3, column=0, padx=5, sticky="w")
        self.entry_comentarios = tk.Entry(eval_frame, width=50, font=("Arial", 14))
        self.entry_comentarios.grid(row=3, column=1, columnspan=5, padx=5, pady=5)

        # Botones
        button_frame = tk.Frame(self.frame_bottom)
        button_frame.pack(pady=10)

        self.btn_prev_image = tk.Button(button_frame, text="Anterior Imagen", command=self.prev_image, font=("Arial", 12))
        self.btn_prev_image.grid(row=0, column=0, padx=10)

        self.btn_save_next = tk.Button(
            button_frame, text="Guardar y Siguiente Carpeta", command=self.save_and_next_folder, font=("Arial", 12)
        )
        self.btn_save_next.grid(row=0, column=1, padx=10)

        self.btn_export = tk.Button(button_frame, text="Exportar Datos", command=self.export_to_csv, font=("Arial", 12))
        self.btn_export.grid(row=0, column=2, padx=10)

        self.btn_next_image = tk.Button(button_frame, text="Siguiente Imagen", command=self.next_image, font=("Arial", 12))
        self.btn_next_image.grid(row=0, column=3, padx=10)

    def select_directory(self):
        directory = filedialog.askdirectory(title="Selecciona un directorio")
        if directory:
            self.load_folders(directory)
        else:
            messagebox.showinfo("Información", "No seleccionaste ningún directorio.")

    def load_folders(self, directory):
        self.image_paths.clear()
        self.image_titles.clear()
        self.current_folder_index = 0

        output_dir = os.path.join(directory, "OUTPUT")
        if not os.path.exists(output_dir):
            messagebox.showerror("Error", "No se encontró la carpeta OUTPUT en el directorio seleccionado.")
            return

        for folder in os.listdir(output_dir):
            folder_path = os.path.join(output_dir, folder)
            frames_path = os.path.join(folder_path, "frames")

            if os.path.isdir(frames_path):
                for subfolder in os.listdir(frames_path):
                    subfolder_path = os.path.join(frames_path, subfolder)
                    if os.path.isdir(subfolder_path):
                        images = [
                            os.path.join(subfolder_path, img)
                            for img in os.listdir(subfolder_path)
                            if img.lower().endswith((".jpg", ".png", ".jpeg"))
                        ]
                        if images:
                            self.image_paths.append(images)
                            self.image_titles.append(f"{folder}/{subfolder}")

        if self.image_paths:
            self.current_image_index = 0
            self.load_subfolder_images(0)
            self.btn_save_next.config(state=tk.NORMAL)
        else:
            messagebox.showinfo("Información", "No se encontraron imágenes en el directorio especificado.")

    def load_subfolder_images(self, folder_index):
        if 0 <= folder_index < len(self.image_paths):
            self.current_subfolder_images = self.image_paths[folder_index]
            self.current_image_index = 0
            self.show_image()

    def show_image(self):
        if 0 <= self.current_image_index < len(self.current_subfolder_images):
            image_path = self.current_subfolder_images[self.current_image_index]
            image_title = self.image_titles[self.current_folder_index]

            image = Image.open(image_path)
            image.thumbnail((1600, 750))
            image_tk = ImageTk.PhotoImage(image)

            self.canvas.delete("all")
            self.canvas.create_image(self.canvas.winfo_width() // 2, self.canvas.winfo_height() // 2, image=image_tk, anchor="center")
            self.canvas.image = image_tk

            self.label_image_title.config(text=f"{image_title} - Imagen {self.current_image_index + 1} de {len(self.current_subfolder_images)}")

    def prev_image(self):
        if self.current_image_index > 0:
            self.current_image_index -= 1
            self.show_image()

    def next_image(self):
        if self.current_image_index < len(self.current_subfolder_images) - 1:
            self.current_image_index += 1
            self.show_image()

    def save_and_next_folder(self):
        visibilidad = self.visibilidad_var.get()
        estado = self.estado_var.get()
        vandalismo = self.vandalismo_var.get()
        comentarios = self.entry_comentarios.get()

        current_path = self.current_subfolder_images[self.current_image_index]
        self.evaluations.append({
            "Carpeta": self.image_titles[self.current_folder_index],
            "Imagen Seleccionada": os.path.basename(current_path),
            "Ruta": current_path,
            "Visibilidad": visibilidad,
            "Estado": estado,
            "Vandalismo": vandalismo,
            "Comentarios": comentarios
        })

        if self.current_folder_index < len(self.image_paths) - 1:
            self.current_folder_index += 1
            self.load_subfolder_images(self.current_folder_index)
        else:
            self.btn_save_next.config(state=tk.DISABLED)
            messagebox.showinfo("Finalizado", "Has evaluado todas las carpetas.")

    def export_to_csv(self):
        if not self.evaluations:
            messagebox.showerror("Error", "No hay evaluaciones para exportar.")
            return

        df = pd.DataFrame(self.evaluations)
        export_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if export_path:
            df.to_csv(export_path, index=False)
            messagebox.showinfo("Exportado", f"Evaluaciones exportadas a {export_path}")
    

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageEvaluatorApp(root)
    root.mainloop()
